<a href="https://colab.research.google.com/github/yinqii/homicide-investigation-project/blob/main/Writeup_CIS545_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CIS 5450 Final Project: Investigating Homicides in America**

*Yinqi Chen, Sneha Parthasarathy, Isabelle Gu*

# Part 1. Introduction



The United States faces one of the lowest homicide clearance rates among industrialized nations, raising serious concerns for both public safety and the justice system. Despite advances in technology and increased resources, thousands of murder cases remain unsolved each year. Understanding the factors that influence whether a homicide is solved is essential for improving investigative practices and optimizing the allocation of law enforcement resources. Our final project aims to identify the drivers behind homicide clearance outcomes in the U.S. by analyzing detailed records from the Murder Accountability Project (MAP), which provides crime-level data on victim demographics, geographic location, weapon type, agency characteristics, and timing of the homicide. To supplement this dataset, we incorporate socioeconomic variables at the state level, such as median income, educational attainment, and population size from the American Community Survey (ACS). This will allow us to explore potential correlations between socioeconomic factors and crime resolution.

Our objectives for this project include identifying patterns in crime solvability across time, demographic groups, and geographic locations, building predictive models to estimate the likelihood that a case will be solved, and exploring potential disparities that may suggest systemic biases or investigative challenges. Predicting which cases are likely to be solved is crucial for improving investigative efficiency. By identifying these cases early, law enforcement can better prioritize resources, focusing on those with the highest probability of resolution. Accurate predictions also enable more effective resource allocation, ensuring that limited manpower and time are directed toward the most solvable cases, while more challenging or unsolved cases can receive additional attention. Moreover, predicting solved cases aids in shaping policy development and refining investigative strategies by highlighting key factors that influence case resolution, such as victim demographics or crime types. This knowledge can inform targeted policing strategies, improve case-solving techniques, and ultimately contribute to a higher clearance rate, enhancing public safety and trust in the justice system.

Our analysis combines exploratory data analysis (EDA), machine learning modeling, and hypothesis testing to uncover the factors that drive case resolution. We aim to build a classification model capable of predicting whether a homicide will be solved, and to identify which crime-level features most significantly influence this outcome. Key findings from our EDA suggest disparities in homicide clearance across victim race and gender, pointing to evidence of systemic biases. We confirm that some of these disparities are statistically significant through hypothesis testing. Interestingly, we also find that geographic variation plays a key role in predicting homicide clearance, indicating significant regional disparities in investigation practices. The insights generated from this analysis will not only inform more efficient resource allocation strategies for law enforcement but also shed light on potential biases in homicide clearance rates, such as disparities across racial and gender lines.


# Part 2. Data Loading and Preprocessing

We begin our analysis by loading the primary dataset from the Murder Accountability Project (MAP), which contains detailed records of homicide cases across the United States. MAP was founded by Thomas Hargrove in 2015, and provides a comprehensive record of U.S. homicides from 1976 to recent years, primarily based on FBI’s Supplementary Homicide Reports (SHR). We obtained the dataset from Kaggle, which contains information on all homicides from 1980 to 2014. Each row in the dataset represents an individual homicide, with variables covering victim demographics (e.g., age, sex, race), crime details (e.g., weapon used, agency type, time period), and whether the case was ultimately solved. The 2014 ending year mitigates concerns that cases are unsolved just because they happened recently.

First, we import the libraries relevant for our report, encompassing both exploratory data analysis and machine learning/modeling. Namely, we import sklearn, pandas, numpy, torch, and matplotlib.   

In [ ]:
# basic data processing and visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns


# libraries for machine learning/modeling tasks (later)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import log_loss, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import random
import math
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost import XGBClassifier
from scipy import stats
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## 2.1 Loading and Preprocessing the MAP Data


### 2.1.1 Loading in MAP Data and Converting to Dataframe

Given that our dataset was of a manageable size, we saved it in our working Google Drive directory and accessed it by mounting Drive to the Colab environment. Mounting allows persistent file storage across sessions without needing to manually re-upload. After configuring the Kaggle API, we downloaded the Murder Accountability Project dataset, unzipped the files, and loaded the primary homicide data (database.csv) into a Pandas dataframe. A preview of the first few rows confirmed successful loading.




In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create the kaggle directory and
# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle
# Read the uploaded kaggle.json file
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
# Download dataset (DO NOT CHANGE)
!kaggle datasets download -d murderaccountability/homicide-reports
!unzip homicide-reports.zip

# Load in data
murder_accountability = pd.read_csv('database.csv')
murder_accountability.head(5)

ValueError: mount failed

### 2.1.2 Analyzing the Data Structure

To gain a better understanding of the structure and distribution of the dataset, we first examine the columns in our dataset. We then inspect basic measures of central tendency and range to identify any unreasonable values, skewness, or outliers in the Murder Accountability Project (MAP) data.

In [ ]:
print(murder_accountability.dtypes)

In [ ]:
print(murder_accountability.shape)

In [ ]:
murder_accountability.describe()

**Key Takeaways**


*  The dataset spans homicides from 1980 to 2014, with a median incident year around 1995. This suggests that there were more homicides in the earlier part of the sample period.

* The median victim age is approximately 30 years, suggesting victims tend to be younger. The maximum victim age is 998, which is clearly unreasonable. Upon further investigation, we realized that the FBI records 998 when the victim's age is not known. This needs to be handled in the data cleaning section.

* Most records involve single-victim, single-perpetrator incidents: the median number of perpetrators and victims per case is zero (suggesting that most incidents only recorded one primary victim, rather than something like a terrorist attack which would have multiple victims).

* Record ID seems to take on a unique value for each recorded crime.



We next check the unique categories of our categorical variables to gain familiarity with what they represent.

In [ ]:
# Unique values for Victim Sex
murder_accountability['Victim Sex'].unique()

In [ ]:
# Unique values for Victim Race
murder_accountability['Victim Race'].unique()

In [ ]:
# Unique values for Victim Ethnicity
murder_accountability['Victim Ethnicity'].unique()

In [ ]:
# Unique values for Weapon
murder_accountability['Weapon'].unique()

In [ ]:
# Unique values for Agency Type
murder_accountability['Agency Type'].unique()

Clearly, we will need to handle the "Unknown Values" for Victim Sex and Victim Race in our data cleaning process.

We also check the values for our outcome variable of interest, Crime Solved, to make sure there are no missing or unknown values.

In [ ]:
# Unique values for Crime Solved
murder_accountability['Crime Solved'].unique()

Finally, we check if there are any duplicated rows and any missing values in the data.

In [ ]:
duplicates = murder_accountability.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

In [ ]:
murder_accountability.isna().sum()

### 2.1.3 Data Cleaning

As a first step in our data cleaning process, we remove variables that are irrelevant, redundant, or uninformative for downstream analysis. Specifically, we eliminate columns based on the following principles:


1.  **Columns with Unique Values per Observation:** These variables offer no predictive power or grouping potential, as they uniquely identify rows without capturing patterns or relationships useful for analysis.

*   Record ID: Unique identifier associated with every case
*  Incident: A three-digit number assigned to differentiate cases occurring within the same month—functionally unique when combined with other fields.

2. **Columns Irrelevant to the Specific Crime Event**: These variables describe meta-information about the record rather than characteristics of the crime itself, and thus are excluded.

*   Victim Count: Describes how many other victims are included in the record—not necessary for crime-level analysis.
*   Perpetrator Count: Similar to Victim Count, used for incident-level grouping rather than individual-level prediction.
*   Record Source: Indicates whether the report came from the FBI or was obtained via FOIA—administrative-- detail with no bearing on the crime itself.

3.  **Redundant or Colinear Variables**: These variables are highly collinear with more informative or already-included fields and introduce redundancy.
*  Agency Code:  Strongly correlated with City, but less interpretable.
*  Agency Name: Strongly correlated with City, but less interpretable.


In [ ]:
columns_to_keep = [
    'Agency Type', # Reporting Agency Details
    'Year', 'Month', 'State', 'City', 'Crime Solved',  'Crime Type', # Time and outcome data
    'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',  # Victim features
    'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race', 'Perpetrator Ethnicity',  # Perpetrator features
    'Relationship', 'Weapon'  # Case-specific details
]
murder_accountability_cleaned = murder_accountability[columns_to_keep]

To ensure consistency and enable efficient downstream analysis, we explicitly convert all variables to their appropriate data types. This step ensures that statistical and modeling functions treat variables correctly.

In [ ]:
# Define column groups by type
numeric_cols = [
    'Victim Age', 'Perpetrator Age', 'Year',
]

categorical_cols = [
    'Agency Type', 'City', 'State', 'Month', 'Crime Type', 'Crime Solved',
    'Victim Sex', 'Victim Race', 'Victim Ethnicity',
    'Perpetrator Sex', 'Perpetrator Race', 'Perpetrator Ethnicity',
    'Relationship', 'Weapon',
]

# 1. Convert numeric columns
for col in numeric_cols:
    if col in murder_accountability_cleaned.columns:
        murder_accountability_cleaned.loc[:, col] = pd.to_numeric(murder_accountability_cleaned[col], errors='coerce')

# 2. Convert categorical columns
for col in categorical_cols:
    if col in murder_accountability_cleaned.columns:
        murder_accountability_cleaned.loc[:, col] = murder_accountability_cleaned[col].astype('category')
# Double check dtypes
print("\nColumn types after conversion:")
print(murder_accountability_cleaned.dtypes)

# Drop any missing values (none are shown above)
murder_accountability_cleaned = murder_accountability_cleaned.dropna().reset_index(drop=True)

Although the dataset does not contain many missing values in many columns, this is likely due to preprocessing during FBI compilation or public data formatting, where unknown or missing responses are encoded as categorical values like "Unknown", "Not Reported", or "Other". So, for our categorical variables, we check how often "Unknown" values are reported.


In [ ]:
# Check frequency of 'Unknown' values in key categorical columns
key_categorical_columns = [
    'Weapon', 'Relationship', 'Victim Sex', 'Victim Race', 'Victim Ethnicity',
    'Perpetrator Sex', 'Perpetrator Race', 'Crime Solved'
]

for col in key_categorical_columns:
    print(f"\nValue counts for {col}:")
    print(murder_accountability_cleaned[col].value_counts(dropna=False))

We remove certain rows from the dataset to ensure high-quality observations for analysis. Our criteria are based on examining unique values and patterns of missingness in critical columns:

1. **Remove Rows with Uninformative or Ambiguous Entries (Especially for Victims)**: Some rows contain vague or unknown values that hinder meaningful interpretation and modeling.

*   Victim Sex or Victim Race is "Unknown": These are essential features for demographic-based analysis. Moreover, such information is typically available to police investigators at the time of the incident, regardless of whether the case was solved. We keep cases where victim's ethnicity is unknown, since they make up a larger share of observations, suggesting this information may not be required when filing a report.


*  Weapon == "Unknown": These rows prevent us from distinguishing between different types of homicide methods (e.g., firearm vs. blunt object), which may be important predictors or control variables.


We opt to drop these rows rather than impute values, as doing so would require us to infer details that law enforcement likely did not possess—introducing the risk of bias in our modeling results. Moreover, "Unknown" counts are among the least common categories for all of these features. As a result, dropping rows with unknown values for victim sex, victim race, and weapon should not result in the loss of too many observations.

However, we retain rows with missing or "Unknown" values for perpetrator characteristics, as this information is frequently unavailable in unsolved cases and is not central to our core analysis. Filtering these out would disproportionately eliminate exactly the cases we aim to understand better.


In [ ]:
# Drop Unknown values
murder_accountability_cleaned = murder_accountability_cleaned[
    (murder_accountability_cleaned['Victim Sex'] != 'Unknown') &
    (murder_accountability_cleaned['Victim Race'] != 'Unknown') &
    (murder_accountability_cleaned['Weapon'] != 'Unknown')
]

# Check that dataframe remains large enough
print(murder_accountability_cleaned.shape)

We also verify whether key geographical information—specifically the city and state where the crime occurred—is missing or labeled as "Unknown". Accurate location data is essential for any geographic or jurisdictional analysis, so we assess the frequency of missing or ambiguous entries in these fields.

In [ ]:
unknown_city_count = ((murder_accountability_cleaned['City'] == 'Unknown') |
                      (murder_accountability_cleaned['City'].isna())).sum()

unknown_state_count = ((murder_accountability_cleaned['State'] == 'Unknown') |
                       (murder_accountability_cleaned['State'].isna())).sum()

print(f"\nNumber of rows with unknown or missing City: {unknown_city_count}")
print(f"Number of rows with unknown or missing State: {unknown_state_count}")

2. **Validating Numeric Fields**: In addition to cleaning categorical variables, we also verify the integrity of key numeric fields, especially Victim Age and Perpetrator Age. Although these columns are recorded as numeric types in the dataset, they may contain implausible or invalid values due to data entry errors or inconsistent reporting.

We apply the following constraints to ensure data quality:
*   Victim Age must be greater than 0, as negative or zero values are not meaningful in this context.
*   We also impose a reasonable upper bound of 110 years, which accommodates edge cases while filtering out extreme outliers.

These constraints help ensure that our analysis reflects plausible demographic distributions and avoids distortions caused by erroneous data entries. Importantly, we do not impute or infer missing ages, as this may introduce bias—especially if age is used as a predictive variable. Instead, we remove records that violate these basic plausibility checks.

Although Perpetrator Age should also be in this age range, the FBI may have reported it as a number outside this range when homicides are unsolved. So, we do not impose any filters.


In [ ]:
invalid_victim_age = murder_accountability_cleaned[
    (murder_accountability_cleaned['Victim Age'] < 0) | (murder_accountability_cleaned['Victim Age'] > 110)
].reset_index(drop=True)

invalid_perp_age = murder_accountability_cleaned[
    (murder_accountability_cleaned['Perpetrator Age'] < 0) | (murder_accountability_cleaned['Perpetrator Age'] > 110)
].reset_index(drop=True)

print("Invalid Victim Ages:") # how many invalid rows
print(invalid_victim_age['Victim Age'])

print("\nInvalid Perpetrator Ages:")
print(invalid_perp_age['Perpetrator Age'])

murder_accountability_cleaned = murder_accountability_cleaned[
    ((murder_accountability_cleaned['Victim Age'] >= 0) & (murder_accountability_cleaned['Victim Age'] <= 110))]

We find that the number 998 is repeated many times for Victim Age, signalling a unknown value that we want to filter out. Further, the age 0 is reported several times for perpetrator age, which could also signal a missing value (or unknown).

Finally, we examine the frequency of different crime types:

In [ ]:
# Get crime types count
crime_type_counts = murder_accountability_cleaned['Crime Type'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(crime_type_counts, labels=crime_type_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Crime Type')
plt.axis('equal')
plt.tight_layout()
plt.show()

Since "Manslaughter by Negligence" accounts for only 1.3% of all observations, we remove all rows with this crime type. This category is procedurally and legally distinct from intentional homicides such as "Murder or Manslaughter" and is therefore less relevant to our analysis. After filtering out these cases, we also drop the Crime Type column entirely, as it no longer contains meaningful variation.



In [ ]:
# Remove rows where Crime Type is "Manslaughter by Negligence"
murder_accountability_cleaned = murder_accountability_cleaned[
    murder_accountability_cleaned['Crime Type'] != 'Manslaughter by Negligence'
].reset_index(drop=True)

# Drop the Crime Type column
murder_accountability_cleaned = murder_accountability_cleaned.drop(columns=['Crime Type'])

print(f"Remaining observations: {len(murder_accountability_cleaned)}")

Now, we print out the shape of our cleaned data set. We still have over 500K observations in our data. So, ensure our analysis reflects the modern homicide landscape, we restrict the dataset to crimes that occurred after 1990. These cases are presumed to be more representative of current social, legal, and technological conditions affecting homicide patterns.

In [ ]:
# Keep only crimes that occurred after 1990
murder_accountability_cleaned = murder_accountability_cleaned[murder_accountability_cleaned['Year'] >= 1990].reset_index(drop=True)

# Confirm enough rows in dataset
print(f"Remaining observations: {len(murder_accountability_cleaned)}")
print(f"Year range: {murder_accountability_cleaned['Year'].min()} to {murder_accountability_cleaned['Year'].max()}")


### 2.1.4 Data Preparation


For ease of analysis, we create some new features to better understand the homicide.

1. Firearm: A binary indicator for whether the perpetrator used a firearm or not.
*    Firearm: 1 if a "Weapon" == Firearm, Shotgun, Gun, Handgun or Rifle; 0 otherwise

2. Census_Region: Grouping 51 States into the following regions (based on United States Census classifications).

*   Northeast: Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont, New Jersey, New York, Pennsylvania
*   Midwest: Illinois, Indiana, Iowa, Kansas, Michigan, Minnesota, Missouri, Nebraska, North Dakota, Ohio, South Dakota, Wisconsin
*   South: Alabama, Arkansas, Delaware, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, West Virginia, District of Columbia
*   West: Alaska, Arizona, California, Colorado, Hawaii, Idaho, Montana, Nevada, New Mexico, Oregon, Utah, Washington, Wyoming

3. Young: A indicator variable for whether the victim is under 30 years old, the median of the original dataset.

In [ ]:
# List of unique weapon types to build definition of firearm
weapon_counts = murder_accountability_cleaned['Weapon'].value_counts()
print(weapon_counts)

In [ ]:
# -------------------------------
# 1. Firearm Indicator
# -------------------------------

firearm_weapons = ['Firearm', 'Gun', 'Handgun', 'Shotgun', 'Rifle']
murder_accountability_cleaned['Firearm'] = murder_accountability_cleaned['Weapon'].isin(firearm_weapons).astype(int)

# -------------------------------
# 2. U.S. Census Region
# -------------------------------
state_region_map = {
    # Northeast
    'Connecticut': 'Northeast', 'Maine': 'Northeast', 'Massachusetts': 'Northeast',
    'New Hampshire': 'Northeast', 'Rhodes Island': 'Northeast', 'Vermont': 'Northeast', # we unconvered later that Rhode Island is misspelled in the MAP data
    'New Jersey': 'Northeast', 'New York': 'Northeast', 'Pennsylvania': 'Northeast',

    # Midwest
    'Illinois': 'Midwest', 'Indiana': 'Midwest', 'Iowa': 'Midwest', 'Kansas': 'Midwest',
    'Michigan': 'Midwest', 'Minnesota': 'Midwest', 'Missouri': 'Midwest',
    'Nebraska': 'Midwest', 'North Dakota': 'Midwest', 'Ohio': 'Midwest',
    'South Dakota': 'Midwest', 'Wisconsin': 'Midwest',

    # South
    'Alabama': 'South', 'Arkansas': 'South', 'Delaware': 'South', 'Florida': 'South',
    'Georgia': 'South', 'Kentucky': 'South', 'Louisiana': 'South', 'Maryland': 'South',
    'Mississippi': 'South', 'North Carolina': 'South', 'Oklahoma': 'South',
    'South Carolina': 'South', 'Tennessee': 'South', 'Texas': 'South',
    'Virginia': 'South', 'West Virginia': 'South', 'District of Columbia': 'South',

    # West
    'Alaska': 'West', 'Arizona': 'West', 'California': 'West', 'Colorado': 'West',
    'Hawaii': 'West', 'Idaho': 'West', 'Montana': 'West', 'Nevada': 'West',
    'New Mexico': 'West', 'Oregon': 'West', 'Utah': 'West', 'Washington': 'West',
    'Wyoming': 'West'
}

murder_accountability_cleaned['Census_Region'] = murder_accountability_cleaned['State'].map(state_region_map)

# -------------------------------
# 3. Young Indicator
# -------------------------------

murder_accountability_cleaned['Young'] = murder_accountability_cleaned['Victim Age'] <= 30


In [ ]:
# Check the column creation worked
firearm_counts = murder_accountability_cleaned['Firearm'].value_counts()
print(firearm_counts)
region_counts = murder_accountability_cleaned['Census_Region'].value_counts()
print(region_counts)

We also create a date column combining year and month, solely for exploratory data analysis.

In [ ]:
murder_accountability_cleaned['Month'] = murder_accountability_cleaned['Month'].str.title()
murder_accountability_cleaned['Date'] = pd.to_datetime(
    murder_accountability_cleaned['Month'] + " " + murder_accountability_cleaned['Year'].astype(str),
    errors='coerce', format='%B %Y'
)
murder_accountability_cleaned['Date']

## 2.2 Merging with the 2010 U.S Census Data


To enrich the MAP dataset, we supplemented it with state-level socioeconomic indicators based on where and when each crime occurred. This additional data is primarily used for exploratory data analysis, allowing us to better understand regional patterns and identify areas that may face heightened risks of unsolved homicide cases.

To access the external data, we utilized the U.S. Census Bureau’s API via the census and us Python packages, generating an API key to programmatically retrieve variables such as state population, median household income, and educational attainment. We use data from the 2010 American Community Survey. Because the Census API sometimes takes longer in the middle of the day, we saved the resulting dataframe to Google Drive and upload it from there. The API code we used is shown commented out for reference.

We pull data on 3 variables, all measured in 2010:
* Population_2010: State-Level population-- important because more poulated states may see more crimes/ busier departments
* Income_2010: Median income-- less well-off states may allocate less resources to crimes
* High_School_2010: The percent of population with at least a high school degree-- captures overall education levels.

In [ ]:
'''
!pip install census us
from census import Census
from us import states

api_key = "43b15e160ca65184a29791495b8c871ffb033d59"
c = Census(api_key)
'''

In [ ]:
'''
fields = [
    'NAME',           # State name
    'B06011_001E',    # Median Household Income
    'B06009_003E',    # Population Total with Education Less than High School Degree
    'B01003_001E'     # Total Population
]
# Make a request
census_data = c.acs5.state(fields, Census.ALL, year=2010)
census_data = pd.DataFrame(census_data)

# Rename columns
census_data = census_data.rename(columns={
    'NAME': 'State',
    'B06011_001E': 'Income_2010',
    'B06009_003E': 'Less_than_HS_2010',
    'B01003_001E': 'Population_2010'
})

# Compute HS_Percent
census_data['HS_Percent_2010'] = (1 - (pd.to_numeric(census_data['Less_than_HS_2010']) / pd.to_numeric(census_data['Population_2010']))) * 100
census_data = census_data.drop(columns=['state'])
'''

In [ ]:
census_data = pd.read_csv('drive/MyDrive/CIS5450/census_data.csv')
census_data = census_data[['State', 'Population_2010', 'Income_2010', 'HS_Percent_2010']]

We examine the "State" variable in both the census data and the cleaned MAP data to ensure we merge correctly.

In [ ]:
print(murder_accountability_cleaned['State'].unique())

In [ ]:
print(census_data['State'].unique())

Note that the state "Rhode Island" is spelled as "Rhodes Island" in the MAP data! We need to fix this before merging.

In [ ]:
# use regex to replace
murder_accountability_cleaned['State'] = murder_accountability_cleaned['State'].replace(r'Rhodes Island', 'Rhode Island', regex=True)

In [ ]:
# Ensure both 'State' columns are strings and consistent
murder_accountability_cleaned['State'] = murder_accountability_cleaned['State'].astype(str)
census_data['State'] = census_data['State'].astype(str)
# Merge: left join to keep all rows in homicide_df, our main dataset of interest
murder_accountability_cleaned = murder_accountability_cleaned.merge(census_data, on='State', how='left')
murder_accountability_cleaned

In [ ]:
# check no mising values
missing_counts = murder_accountability_cleaned.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
print("Columns with missing values:")
print(missing_counts)

# Part 3. Exploratory Data Analysis

Before building predictive models, we conduct an exploratory analysis to better understand the structure and patterns present in the data. We examine the distributions of key variables such as clearance status, victim demographics, weapon type, and agency characteristics. We also investigate potential relationships between clearance outcomes and socioeconomic indicators such as state-level income and educational attainment. Our initial analysis helps identify important trends, detect anomalies or missing values, and guide feature engineering and model design choices in later stages.



In [ ]:
homicide_df = murder_accountability_cleaned.copy() # copy for EDA

## 3.1. Key Variables and their Summary Statistics

We start our exploratory data analysis section by providing some context on the key columns in our dataset. We define each of the features we plan to use in our EDA and subsequent modeling.


In [ ]:
homicide_df.columns

**Key Variables**

- **Agency Type**: Type of law enforcement agency (e.g., municipal police, state police). Likely affects clearance rates due to differences in jurisdiction, resources, and investigative capacity.

- **Year, Month, Date**: Temporal variables used to analyze trends in crime incidence and resolution over time.

- **State, City**: Geographic identifiers capturing location-specific crime patterns. Clearance rates may vary across jurisdictions due to differences in local policy and resource allocation.

- **Crime Solved**: Binary outcome variable indicating whether a crime was solved (`1`) or unsolved (`0`). This serves as the primary dependent variable in the analysis.
- **Victim Sex**: Binary indicator for male victims (`1 = male`).
- **Victim Race**: Categorical variable (`Black`, `White`, `Asian/Pacific Islander`, `Native American/Alaskan Native`).
- **Victim Ethnicity**: Categorical variable (`Hispanic`, `Non-Hispanic`, `Unknown`).
- **Victim Age**: Continuous numeric variable.

- **Weapon**: Categorical variable describing the type of weapon used (e.g., handgun, knife). Weapon type may influence both crime severity and resolution probability.

- **Firearm**: Binary indicator for whether the weapon used was a firearm. Firearm-related homicides may be harder to solve due to the potential for anonymous attacks.

- **Census Region**: Regional classification (`Northeast`, `Midwest`, `South`, `West`) used to examine broader geographic trends.

- **Young**: Binary indicator for whether the victim was under 30. Younger victims may exhibit different victimization and resolution patterns.

- **Population_2010**: State population from the 2010 Census, used to contextualize crime volume and departmental burden.

- **Income_2010**: Median household income by state in 2010, serving as a proxy for local economic conditions.

- **HS_Percent_2010**: Percentage of the state population with at least a high school diploma as of 2010, serving as a proxy for educational attainment.

---

**Additional Variables Used for EDA Only**

We include several variables that would not be known to investigators at the time of the crime. These are excluded from predictive modeling but retained for exploratory data analysis :
- **Perpetrator Sex**: Binary indicator (`1 = male`).
- **Perpetrator Race**: Categorical variable (`Black`, `White`, `Asian/Pacific Islander`, `Native American/Alaskan Native`, `Unknown`).
- **Perpetrator Ethnicity**: Categorical variable (`Hispanic`, `Non-Hispanic`, `Unknown`).
- **Perpetrator Age**: Continuous numeric variable. Coded as `0` when unknown.
- **Relationship**: Relationship between the perpetrator and the victim. Coded as "Unknown" in unsolved cases.

---

These variables capture how the demographic, geographic, and socioeconomic factors influence the likelihood of homicide resolution.

To better understand the structure of the homicide dataset, we examine descriptive statistics for key numeric variables. These variables capture characteristics of the crime event (e.g., use of a firearm, year of occurrence), victim demographics (e.g., sex, race, ethnicity, and age), and the socioeconomic environment (e.g., state-level income and education rates). Additionally, we introduce binary indicators for whether the homicide was solved, whether the victim was male, whether the victim was white, and whether the victim was Hispanic.



In [ ]:
# Key Summary Statistics with Cleaned Data
homicide_df['Crime Solved Binary'] = homicide_df['Crime Solved'].map({'Yes': 1, 'No': 0})
homicide_df['Victim_Male'] = homicide_df['Victim Sex'].map({'Male': 1, 'Female': 0})
homicide_df['Victim_White'] = homicide_df['Victim Race'].apply(lambda x: 1 if x == 'White' else 0)
homicide_df['Victim_Black'] = homicide_df['Victim Race'].apply(lambda x: 1 if x == 'Black' else 0)
homicide_df['Victim_Asian'] = homicide_df['Victim Race'].apply(lambda x: 1 if x == 'Asian/Pacific Islander' else 0)
homicide_df['Victim_Native_American'] = homicide_df['Victim Race'].apply(lambda x: 1 if x == 'Native American/Alaska Native' else 0)

homicide_df['Victim_Hispanic'] = homicide_df['Victim Ethnicity'].apply(lambda x: 1 if x == 'Hispanic' else 0)

# Select important numeric variables
numeric_vars = ['Year', 'Victim Age', 'Firearm', 'Population_2010', 'Income_2010', 'HS_Percent_2010', 'Crime Solved Binary', 'Victim_Male', 'Victim_White', 'Victim_Black', 'Victim_Asian', 'Victim_Native_American', 'Victim_Hispanic']
numeric_summary = (
    homicide_df[numeric_vars]
    .describe()
    .T
    .round(2)
)

numeric_summary

**Key Takeaways**

*  The dataset spans homicides occurring between 1990 and 2014, with a median incident year of 2000, suggesting that more homicides occured in the earlier part of our sample period.
*   Victims are relatively young on average, with a median victim age of 29 years, but there is a wide age range from infancy to nearly 100 years old. Although some extreme values exist (e.g., very young or very old victims), we chose not to remove these observations to preserve the real-world variation inherent to homicide data, ensuring that rare but important cases are not excluded from analysis.
*   Firearms are involved in approximately 72% of homicide cases, highlighting their prevalence as a weapon.
*   About 69% of homicide cases are cleared (solved), while roughly 31% remain unsolved. This suggests that our classes (solved vs unsolved crimes) will show some imbalance.
*   The majority of victims are male (79%) and exactly half of the victims are Black (50%), while 48% of victims are white. Victims who are Asian/Pacific Islander and Native American/Alaska Native make up a much smaller portion of our data (less than 3% combined)
*  12% of victims are identified as Hispanic.
* State-level socioeconomic indicators show substantial variation, with median household incomes around $26,249 and an average high school attainment rate of about 81%.







While the summary table and EDA visualizations below capture the proportion of homicides that involve victims of various demographic types and certain weapons, we provide a histogram to illustrate the distribution of victim age, the only crime-level continous feature in the dataset.  

In [ ]:
# Victim age distribution
plt.figure(figsize=(10, 4))
sns.histplot(homicide_df['Victim Age'], bins=30, kde=True)
plt.title('Distribution of Victim Age')
plt.show()

The histogram reveals that the majority of crimes involve victims between the ages of 15 and 40. However, there are noticeable local peaks at both extremes of the distribution—specifically among victims aged 0–5 and 95–100. This distribution is moderately skewed, suggesting that scaling the Victim Age variable may be necessary for subsequent modeling to ensure more stable results.

## 3.2 Geographic Variation in Homicides Across the United States

### 3.2.1 Homicide Rates Across the United States

To explore spatial variation in homicide activity across the United States, we plot the number of homicides by state using a choropleth map. Visualizing the data geographically enables a more intuitive understanding of where homicide cases are concentrated relative to population size. Rather than using bar charts or line charts, which can obscure regional patterns and are less interpretable for state-level comparisons, a choropleth map provides an immediate, visual representation of how homicide rates vary across the country. This format highlights the geographic clustering of homicide incidence, allowing us to easily identify hotspots and regional disparities at a glance.

In [ ]:
! pip install plotly

import plotly.express as px

# aggregate counts by state
state_counts = homicide_df['State'].value_counts().reset_index()
state_counts.columns = ['State', 'Homicide Count']

# map full state names to their abbreviations (required for map on plotly)
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}
state_counts['State Abbrev'] = state_counts['State'].map(us_state_abbrev)

# plot using plotly
fig = px.choropleth(
    state_counts,
    locations='State Abbrev',
    locationmode="USA-states",
    color='Homicide Count',
    color_continuous_scale="Reds",
    scope="usa",
    title='Homicide Counts by U.S. State in Our Sample Period (1990-2014)'
)
fig.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

We can see that the total number of homicides recorded in our dataset (1990–2014) varies significantly by state. California has a shockingly high number of homicides compared to the rest of the country. However, because total homicide counts can be heavily influenced by population size, we supplement this view by plotting the number of homicides per 100,000 residents using 2010 American Community Survey (U.S. Census) population estimates. This per capita adjustment offers a clearer picture of where homicide risks are disproportionately high relative to state population, providing more meaningful comparisons across states of different sizes.

In [ ]:
state_counts = homicide_df['State'].value_counts().reset_index()
state_counts.columns = ['State', 'Homicide Count']

state_counts = state_counts.merge(census_data[['State', 'Population_2010']], on='State', how='left')
state_counts['Homicides per 100k'] = (state_counts['Homicide Count'] / state_counts['Population_2010']) * 100000

state_counts['State Abbrev'] = state_counts['State'].map(us_state_abbrev)

fig = px.choropleth(
    state_counts,
    locations='State Abbrev',
    locationmode="USA-states",
    color='Homicides per 100k',
    color_continuous_scale="OrRd",
    scope="usa",
    title='Homicides per 100,000 Residents by U.S. State (1990–2014) using 2010 Population Totals'
)

fig.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

After normalizing by state population, homicide rates across the United States appear more evenly distributed. The highest per capita homicide rates are concentrated in the South, while the Midwest generally exhibits lower homicide rates per capita. Areas with urban centers, such as New York and the D.C. metro area show slightly higher rates of homicide, even after normalizing for the population. This visualization highlights the advantage of using a choropleth map, as it allows clear regional patterns to emerge that would be less immediately apparent with bar charts or other formats.


### 3.2.2 Clearance Rates Across the United States

We also plot Clearance rates (the proportion of the crimes solved) for each state on a chloropleth map.

In [ ]:
# compute state clearance rates
clearance_by_state = homicide_df.groupby('State')['Crime Solved Binary'].mean().reset_index()
clearance_by_state.columns = ['State', 'Clearance Rate']
clearance_by_state['State Abbrev'] = clearance_by_state['State'].map(us_state_abbrev)

# plot with Plotly
fig = px.choropleth(
    clearance_by_state,
    locations='State Abbrev',
    locationmode="USA-states",
    color='Clearance Rate',
    color_continuous_scale="Blues",
    scope="usa",
    title='Homicide Clearance Rate by U.S. State (Proportion Solved)'
)

fig.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

We can see significant regional variation in the proportion of homicides that are solved across the United States. The Northwest states seem to have a higher clearance rates, while Northeast states, especially New York and Illinois have a significantly lower clearance rate. Interestingly, states that have densely populated urban centers, such as New York and California, exhibit lower clearance rates. This plot highlights the significant regional variation in homicide clearance across the United States, with some states having as low as 50% of all homicides solved, and others over 90%.

To better understand the geographic variation in homicide clearance rates across the United States, we examine the relationship between state-level socioeconomic characteristics — such as median income, educational attainment, and population size — and the average clearance proportion observed in each state. We use the data from the 2010 American Community Survey to capture socioeconomic disparities between states.

We plot the mean clearance rate by state against state level socioeconomic data using scatter plots, allowing us to see if state socioeconomic conditions can explain a substantial part of the regional variation in homicide clearance shown in the above figures.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

state_data = clearance_by_state.merge(census_data, on='State')

# Plot 1: Clearance Rate vs Median Income
sns.regplot(data=state_data, x='Income_2010', y='Clearance Rate', ax=axes[0])
axes[0].set_title('Clearance Rate vs. Median Income')
axes[0].set_xlabel('Median Income')
axes[0].set_ylabel('Clearance Rate')
axes[0].grid(True)

# Plot 2: Clearance Rate vs Population
sns.regplot(data=state_data, x='Population_2010', y='Clearance Rate', ax=axes[1])
axes[1].set_title('Clearance Rate vs. State Population')
axes[1].set_xlabel('Population (2010)')
axes[1].set_ylabel('')
axes[1].grid(True)

# Plot 3: Clearance Rate vs High School Percentage
sns.regplot(data=state_data, x='HS_Percent_2010', y='Clearance Rate', ax=axes[2])
axes[2].set_title('Clearance Rate vs. High School Attainment')
axes[2].set_xlabel('HS Percent (2010)')
axes[2].set_ylabel('')
axes[2].grid(True)
plt.tight_layout()
plt.show()


First, we find a negative association between median household income and homicide clearance rates: states with higher median incomes tend to exhibit slightly lower clearance proportions. This could be due to states with higher income also having more population density in urban centers, potentially leading to overburdened law enforcement agencies.

Second, we observe a negative relationship between state population size and clearance rate. However, the pattern appears nonlinear, with large-population states (e.g., California, Texas) exerting disproportionate influence. To address this, we apply a log transformation below to the state population variable, which reveals a clearer downward trend.

Finally, we find a negative relationship between high school educational attainment and clearance rates: states with higher rates of high school graduation tend to solve a lower share of homicide cases. This aligns with the relationship between income and homicide clearance, suggesting that socioeconomic conditions are slightly negatively correlated with homicide clearance.

Overall, these patterns suggest that both economic and educational factors at the state level may play a role in shaping the geographic/regional variation in homicide investigation outcomes.

In [ ]:
# Log transform population
state_data['Log_Population_2010'] = np.log1p(state_data['Population_2010'])
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Clearance Rate vs Median Income
sns.regplot(data=state_data, x='Income_2010', y='Clearance Rate', ax=axes[0])
axes[0].set_title('Clearance Rate vs. Median Income')
axes[0].set_xlabel('Median Income')
axes[0].set_ylabel('Clearance Rate')
axes[0].grid(True)

# Plot 2: Clearance Rate vs Log Population
sns.regplot(data=state_data, x='Log_Population_2010', y='Clearance Rate', ax=axes[1])
axes[1].set_title('Clearance Rate vs. Log State Population')
axes[1].set_xlabel('Log(Population + 1)')
axes[1].set_ylabel('')
axes[1].grid(True)

# Plot 3: Clearance Rate vs High School Percentage
sns.regplot(data=state_data, x='HS_Percent_2010', y='Clearance Rate', ax=axes[2])
axes[2].set_title('Clearance Rate vs. High School Attainment')
axes[2].set_xlabel('HS Percent (2010)')
axes[2].set_ylabel('')
axes[2].grid(True)
plt.tight_layout()
plt.show()

Once we applied the log transformation, the relationship between state-level clearance rates and the log of state population looks more linear, and is clearly a negative association.


## 3.3 Temporal Variation in Homicides and Clearance Rates

### 3.3.1 Homicides by U.S. Census Region Over Time.

In this section, we analyze how the number of homicides has evolved over time from 1990 to 2014 for each U.S. region. To visualize this pattern, we use a line chart, which captures trends over time. We aggregate homicide cases at a monthly frequency to reveal seasonal variation while smoothing out fluctuations that may occur at a daily frequency. First, we how trends in the number of homicides by U.S. Census Region, capturing both overall trends in manslaughter as well as variation within each region.



In [ ]:
# plot homicides by region over time with line chart
region_trend = (
    homicide_df
    .groupby([pd.Grouper(key='Date', freq='ME'), 'Census_Region'])
    .size()
    .reset_index(name='Homicide Count')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=region_trend, x='Date', y='Homicide Count', hue='Census_Region')
plt.title('Monthly Homicide Trends by U.S. Census Region (1990–2014)', fontsize=18)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Homicides', fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.show()

Based on the line chart, the number of homicides has decreased over time for all Census Regions, with the sharpest decreasing occuring in the South. Generally, the Northeast reports the fewest homicides while the South reports the most (though this visualization does not account for population). There are also significant fluctuations in the number of homicides reported within a given year. The South has seen the largest decline in homicides over time, as well as the largest fluctuations, while trends for the Midwest appear slightly more stable.

### 3.3.2 Clearance Rates by U.S. Region, Victim Race, Weapon, and Agency Type Over Time

Next, we examine the evolution of homicide clearance rates over time, focusing on regional variations across the U.S., as well as disparities by victim race, weapon type, and agency type.

Given that crime clearance is our primary outcome of interest, it is essential to explore how it varies across key demographic factors and characteristics of the crime itself. By incorporating victim race into the analysis, we aim to investigate potential racial disparities in the likelihood of solving a homicide. Additionally, analyzing clearance rates by weapon type allows us to assess whether the nature of the crime influences the probability of case resolution over time. In addition, we analyze clearance rates by agency type, as differences in resources, investigative practices, or institutional priorities between local, state, tribal, and federal agencies may systematically affect case outcomes. Agencies with greater investigative capacity or specialized resources may achieve higher clearance rates, while agencies with limited staff, funding, or jurisdictional authority could face lower solve rates.

To capture these temporal trends, we use line charts. However, rather than aggregating the data on a monthly basis, we choose to aggregate it yearly. This is because the clearance rates, which are proportions, tend to exhibit a narrower range of small (less than 1) values on a monthly scale. Consequently, reporting results at a monthly level may capture too many fluctuations and potentially obscure broader trends. By using annual data, we better capture the underlying temporal patterns and ensure a more meaningful analysis.

In [ ]:
clearance_by_region = (
    homicide_df
    .groupby(['Year', 'Census_Region'])['Crime Solved Binary']
    .mean()
    .reset_index(name='Clearance Rate')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=clearance_by_region, x='Year', y='Clearance Rate', hue='Census_Region', marker='o')
plt.title('Homicide Clearance Rate by U.S. Census Region Over Time (1990-2014)', fontsize = 18)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Share of Solved Homicides', fontsize=14)
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.show()

We can see that Southern states consistently have higher clearance rates on average, even though they report the most homicides. Clearance rates in the Northeast improved from 1990 to 2006, before falling again towards the ed of our sample period. The majority of crimes are solved, with clearance rates generally falling between the 0.5 to 0.8 range for all Census regions.

In [ ]:
clearance_by_race = (
    homicide_df
    .groupby(['Year', 'Victim Race'])['Crime Solved Binary']
    .mean()
    .reset_index(name='Clearance Rate')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=clearance_by_race, x='Year', y='Clearance Rate', hue='Victim Race', marker='o')
plt.title('Homicide Clearance Rate by Victim Race Over Time (1990-2014)', fontsize = 18)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Proportion of Cases Solved', fontsize = 14)
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.show()


Interestingly, we can see significant variation in homicide clearance rates based on the victim's race. Overall, crimes that involve Black victims have lower clearance rates, consistently below 0.7. Clearance rates for victims who identify as Native American/Alaska Native have risen over time. In general, clearance rates for white victims tend to be higher than for Black and Asian victims, suggesting some evidence of racial discrepancies in homicide clearance.


In [ ]:
clearance_by_weapon = (
    homicide_df
    .groupby(['Year', 'Firearm'])['Crime Solved Binary']
    .mean()
    .reset_index(name='Clearance Rate')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=clearance_by_weapon, x='Year', y='Clearance Rate', hue='Firearm', marker='o')
plt.title('Homicide Clearance Rate by Weapon Over Time (1990-2014)', fontsize = 18)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Proportion of Cases Solved', fontsize = 14)
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.show()


Interestingly, the discrepancy in clearance rates between crimes committed with a firearm weapon and crimes committed with other weapons has increased over time. In general, crimes committed with a firearm are less likely to be solved, and their clearance rate has slightly fallen over time. Crimes committed with other weapons are more likely to be solved, and their clearance rate has risen slightly over time.

In [ ]:
clearance_by_agency_type = (
    homicide_df
    .groupby(['Year', 'Agency Type'])['Crime Solved Binary']
    .mean()
    .reset_index(name='Clearance Rate')
)

plt.figure(figsize=(14, 6))
sns.lineplot(data=clearance_by_agency_type, x='Year', y='Clearance Rate', hue='Agency Type', marker='o')
plt.title('Homicide Clearance Rate by Agency Type Over Time (1990-2014)', fontsize = 18)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Proportion of Cases Solved', fontsize = 14)
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.show()

When examining homicide clearance rates by agency type, we observe significant fluctuations over time and meaningful variation across different types of law enforcement agencies.

Municipal police departments generally exhibit lower clearance rates compared to sheriffs' offices and state police. Tribal police agencies, although fewer in number, consistently display extremely high clearance rates, maintaining a solve rate of 1.0 throughout the sample period. Regional police departments and special police units show substantial year-to-year volatility in clearance rates, suggesting greater variability in investigative capacity, caseload composition, or reporting practices within these smaller or specialized agencies. These findings highlight the importance of agency characteristics in influencing homicide investigation outcomes, with broader implications for understanding resource disparities across different types of law enforcement.

## 3.4 Victim Age and Clearance Rate

We also analyze clearance rates by the victim’s age. Because age is a continuous numeric variable, a traditional line chart would not be appropriate, as there are effectively an infinite number of unique values. To address this, we bin victim ages into 5-year intervals and calculate the average clearance rate within each bin. This approach smooths the data, allowing us to better identify broader patterns in how clearance rates vary across different age groups without being overly sensitive to small fluctuations. Since we are no longer examining temporal data, we choose to use a bar chart.



In [ ]:
# Bin victim ages
homicide_df['Victim_Age_Bin'] = pd.cut(homicide_df['Victim Age'], bins=range(0, 115, 5), right=False)

#Group by age bin and compute clearance rate
age_clearance = (
    homicide_df
    .groupby('Victim_Age_Bin', observed=True)['Crime Solved Binary']
    .mean()
    .reset_index(name='Clearance Rate')
)
age_clearance['Age Bin Label'] = age_clearance['Victim_Age_Bin'].astype(str)

# Plot as a bar chart
plt.figure(figsize=(14, 5))
sns.barplot(data=age_clearance, x='Age Bin Label', y='Clearance Rate', color='skyblue')
plt.title('Homicide Clearance Rate by Victim Age (5-Year Bins)', fontsize=18)
plt.xlabel('Victim Age Group', fontsize=14)
plt.ylabel('Proportion of Cases Solved', fontsize=14)
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


Homicide clearance rates are highest among the youngest victims and lowest for victims between the ages of 15 and 30. Clearance rates then gradually increase for older age groups, before declining again among victims aged 95 to 100. This pattern suggests that cases involving very young and older victims may receive more investigative attention or be easier to resolve, while homicides involving young adults are comparatively less likely to be solved.



## 3.5 Victim and Perpetrator Relationships

In this section, we examine how homicide patterns vary according to the characteristics of victims and perpetrators recorded in the dataset.

Specifically, we analyze the distribution of homicide incidents across different victim and perpetrator demographic groups. We focus on race and sex categories for both victims and perpetrators, and create interaction terms to explore how different combinations of characteristics relate to homicide patterns.

To visualize these relationships, we construct a heatmap showing the total number of homicides committed by perpetrators of a given category against victims of a corresponding category. A heatmap is particularly well-suited for this analysis because it allows for the simultaneous visualization of two categorical variables (victim group and perpetrator group) and the intensity of their interactions. Unlike separate bar charts or tables, the heatmap format clearly highlights which demographic pairings are most frequent, revealing concentration patterns and asymmetries in an interpretable way.

We do not include victims who are Asian/Pacific Islander or Native American/Alaskan Native, as they make up a small portion of the dataset.

In [ ]:
excluded_races = ["Asian/Pacific Islander", "Native American/Alaska Native"]

filtered_df = homicide_df[
    ~homicide_df['Victim Race'].isin(excluded_races) &
    ~homicide_df['Perpetrator Race'].isin(excluded_races)
].copy()

# Combine race and sex for both victim and perpetrator (interaction)
filtered_df['Victim Group'] = filtered_df['Victim Race'].astype(str) + ' - ' + filtered_df['Victim Sex'].astype(str)
filtered_df['Perpetrator Group'] = filtered_df['Perpetrator Race'].astype(str) + ' - ' + filtered_df['Perpetrator Sex'].astype(str)

# Count occurrences of each Victim-Perpetrator pair
heatmap_data = pd.crosstab(filtered_df['Victim Group'], filtered_df['Perpetrator Group'])

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='Blues', linewidths=0.5)
plt.title('Homicides by Race & Sex of Victims and Perpetrators')
plt.xlabel('Perpetrator Category')
plt.ylabel('Victim Category')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

We find two interesting observations:

1.  Homicides rarely cross racial lines.
The heatmap shows that perpetrators and victims tend to share the same racial group when the homicide is solved (i.e. perpetrator race is known). In particular, the "Black Male – Black Male" and "White Male – White Male" cells display notably high counts, indicating that most solved homicides occur within rather than across racial groups.

2.  Black male victims are disproportionately associated with unknown perpetrators. The heatmap suggests that cases involving Black male victims are more likely to have unidentified perpetrators compared to other demographic groups. This pattern points to potential racial disparities in the likelihood of solving homicides.



## 3.5 Feature Correlation Heatmap: Numerical Variables

We examine the correlations between our key features and outcome variable with a heatmap. Since a victim is either Black, White, Asian, or Native American (and the latter two categories make up a small share of total observations), we only include the indicator for whether the victim is Black.

In [ ]:
corr_vars = [
    'Crime Solved Binary',
    'Firearm',
    'Victim_Male',
    'Victim_Black',
    'Victim Age',
    'Population_2010',
    'Income_2010',
    'HS_Percent_2010'
]

# Compute correlation matrix
corr_matrix = homicide_df[corr_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    vmin=-1, vmax=1,
    linewidths=0.5,
    square=True,
    cbar_kws={"shrink": 0.8}
)

plt.title('Feature Correlation Matrix', fontsize=18)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


**Key Takeaways**

*   Firearm usage is negatively correlated with homicide clearance. We observe a moderate negative correlation (-0.12) between the use of a firearm and the likelihood of a case being solved.
*   Victim demographics show weak but meaningful patterns. Being a male victim (-0.12) or a Black victim (-0.12) is associated with slightly lower clearance rates, indicating potential demographic disparities in case outcomes. Victim age shows almost no correlation (0.01) with clearance probability.
*   Socioeconomic factors at the state level also matter. Higher state-level median income is negatively correlated with clearance rates (-0.13), while higher state-level high school attainment and population are also slightly negatively correlated with clearance.
*   Strong correlations between predictors themselves. There are strong positive correlations between state population and educational attainment (0.65) and between income and education (0.42), reflecting broader socioeconomic patterns. This state-level data is time-invariant, meaning it is the same for all crimes in a given state, potentially explaining the high correlation. Because of the high correlation between the state indicators, we never include them all in the same model in our subsequent analysis. Instead, we experiment with things like principal component analysis (PCA) to combine the information they provide.


Notably, most of our input features are not highly correlated with each other.

## 3.6 Correlation Between Categorical Variables & Crime Solved

Because the heatmap in the previous section only computes the correlation between numerical features and whether the case is solved, we also implement the [Spearman rank-order correlation metric](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html) for our categorical variables.

In [ ]:
# Correlation between crime solved and race
corr, pval = stats.spearmanr(homicide_df['Crime Solved Binary'], homicide_df['Victim Race'])
print('Correlation between Crime Solved & Victim Race: ' + str(corr))

In [ ]:
# Correlation between crime solved and U.S. State
corr, pval = stats.spearmanr(homicide_df['Crime Solved Binary'], homicide_df['State'])
print('Correlation between Crime Solved & State: ' + str(corr))

In [ ]:
# Correlation between crime solved and weapon
corr, pval = stats.spearmanr(homicide_df['Crime Solved Binary'], homicide_df['Weapon'])
print('Correlation between Crime Solved & Weapon: ' + str(corr))

In [ ]:
# Correlation between crime solved and weapon
corr, pval = stats.spearmanr(homicide_df['Crime Solved Binary'], homicide_df['Agency Type'])
print('Correlation between Crime Solved & Weapon: ' + str(corr))

There do not seem to be any strong correlations between whether a crime is solved and the victim's race, the weapon that was used, the agency type, and the in which state the crime took place.

# Part 4. Hypothesis Testing

Based on the patterns we observed during our exploratory data analysis, there are significant disparities in the homicide clearance rate for victims of different demographic groups, such as victims of different age and of a different race. In this section, we aim to evaluate whether these differences are statistically significant.

## 4.1 Testing the Relationship between Victim Age and Crime Solvability

****

The purpose of this analysis is to evaluate whether there is a significant difference in the mean victim age between homicide cases that are solved and those that remain unsolved. In our exploratory data analysis, we saw that the share of crimes that are solved varies significantly for different age groups. By examining whether unsolved cases tend to involve victims within a particular age range, we aim to understand if victim age influences investigative priorities or follow-up efforts, thereby contributing to the likelihood of case resolution.

**Hypotheses:**

* **H₀**: There is no difference in the mean victim age between solved and unsolved homicide cases.

* **H₁**: There is a significant difference in the mean victim age between solved and unsolved homicide cases.

To test these hypotheses, we use a permutation test. Under the null hypothesis, the victim's age is assumed to be independent of case solvability. By randomly shuffling the victim ages among the solved and unsolved groups and recalculating the difference in means over many iterations, we can generate a null distribution of mean differences. The p-value is then determined as the proportion of permutations in which the simulated difference equals or exceeds the observed difference, indicating whether the observed association is statistically significant.




In [ ]:
# Calculate the Observed Difference

# Find means for each group
solved_victim_ages = homicide_df[homicide_df['Crime Solved Binary'] == 1]['Victim Age'].values
unsolved_victim_ages = homicide_df[homicide_df['Crime Solved Binary'] == 0]['Victim Age'].values

mean_age_solved = np.mean(solved_victim_ages)
mean_age_unsolved = np.mean(unsolved_victim_ages)

observed_diff = mean_age_solved - mean_age_unsolved

print("Observed mean victim age (solved cases):", mean_age_solved)
print("Observed mean victim age (unsolved cases):", mean_age_unsolved)
print("Observed difference (solved - unsolved):", observed_diff)

# Construct Null Distribution

all_victim_ages = homicide_df['Victim Age'].values

# Get number of observations in each group
n_solved = len(solved_victim_ages)
n_iterations = 10000
simulated_diffs = np.empty(n_iterations)

# Permutation testing
np.random.seed(42)
for i in range(n_iterations):
    shuffled = np.random.permutation(all_victim_ages)
    pseudo_solved = shuffled[:n_solved]
    pseudo_unsolved = shuffled[n_solved:]
    simulated_diffs[i] = np.mean(pseudo_solved) - np.mean(pseudo_unsolved)

# Calculate the p-value
p_value = np.mean(np.abs(simulated_diffs) >= np.abs(observed_diff))
print("Permutation Test p-value:", p_value)

#Visualize Null Distribution vs Observed Difference
plt.figure(figsize=(10, 6))
plt.hist(simulated_diffs, bins=50, alpha=0.75)
plt.axvline(observed_diff, color='red', linestyle='dashed',
            label=f'Observed diff = {observed_diff:.2f}')
plt.title("Null Distribution of Mean Victim Age Differences VS Observed Difference")
plt.xlabel("Mean Difference (Solved - Unsolved)")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
print("Mean of simulated differences:", np.mean(simulated_diffs))
print("Standard deviation of simulated differences:", np.std(simulated_diffs))


Our test yielded an observed mean victim age of 33.02 years in solved cases versus 32.52 years in unsolved cases—a difference of 0.50 years. With 10,000 permutations, the p-value was 0.00 (i.e., less than 0.01), meaning that we can reject the null hypothesis. The distribution shows that none of the shuffled simulations produced a mean difference as large as the observed 0.50-year gap. This indicates that the difference in victim age between solved and unsolved cases is statistically significant and not due to chance.

Although a half-year difference may seem minor, its statistical robustness (given our large sample) suggests that older victims are more likely to be associated with solved cases. This finding is relevant in indicating that victim age may influence homicide case outcomes, potentially reflecting differences in investigative priorities or resource allocation. Another possibility to note is that it may also imply that cases involving older victims are more likely to be solved because older victims have had a longer period of exposure to society, leaving more traces or evidence that can aid the investigation.

## 4.2 Testing the Relationship between Victim Race and Crime Solvability

****

The objective of this analysis is to examine whether the clearance rate (proportion of cases solved) differs by victim race, specifically comparing cases with White victims to those with Black victims as the dataset is dominated by White and Black cases. By testing for differences in solve rates across these groups, we can investigate potential biases or systemic disparities in law enforcement practices. If a significant difference is found, it would suggest that victim race may influence how cases are prioritized or investigated.

**Hypotheses:**

- **H₀:** There is no difference in the clearance rate of homicide cases between racial groups (e.g., White and Black victims).
- **H₁:** There is a significant difference in the clearance rate between racial groups.

A permutation test will be used to assess these hypotheses by randomly shuffling the victim race labels and computing the difference in solve rates, thereby generating a null distribution for comparison.

In [ ]:
race_subset = homicide_df[homicide_df['Victim Race'].isin(['White', 'Black'])].copy()

# Observed proportions of solved for each group
white_solved = race_subset[race_subset['Victim Race'] == 'White']['Crime Solved Binary'].mean()
black_solved = race_subset[race_subset['Victim Race'] == 'Black']['Crime Solved Binary'].mean()
observed_diff = white_solved - black_solved

print("Observed White Solve Rate:", round(white_solved, 3))
print("Observed Black Solve Rate:", round(black_solved, 3))
print("Observed Difference (White - Black):", round(observed_diff, 3))

# Permutation test
n_iterations = 10000
simulated_diffs = np.zeros(n_iterations)

# Extract the race labels and the 'Crime Solved Binary' array to shuffle
race_labels = race_subset['Victim Race'].values
solved_array = race_subset['Crime Solved Binary'].values
n_white = sum(race_labels == 'White')

np.random.seed(42)
for i in range(n_iterations):
    shuffled_labels = np.random.permutation(race_labels)

    white_solved_mean = solved_array[shuffled_labels == 'White'].mean()
    black_solved_mean = solved_array[shuffled_labels == 'Black'].mean()

    simulated_diffs[i] = white_solved_mean - black_solved_mean

# Find p-value (two-sided)
p_value = np.mean(np.abs(simulated_diffs) >= np.abs(observed_diff))

print("Permutation Test p-value:", p_value)

In [ ]:
#Visualize Null Distribution vs Observed Difference
plt.figure(figsize=(10, 6))
plt.hist(simulated_diffs, bins=50, alpha=0.75)
plt.axvline(observed_diff, color='red', linestyle='dashed',
            label=f'Observed diff = {observed_diff:.2f}')
plt.title("Null Distribution of Mean Victim Age Differences VS Observed Difference")
plt.xlabel("Mean Difference (Solved - Unsolved)")
plt.ylabel("Frequency")
plt.legend()
plt.show()

The observed clearance rate for homicide cases with White victims is 0.75, compared to 0.638 for cases with Black victims—resulting in an observed difference (White − Black) of 0.112. With 10,000 permutations, the p-value was 0.00 (i.e., less than 0.01), indicating that none of the random shuffles produced a clearance rate difference as large as 0.112. This strongly suggests that the difference in solve rates between White and Black victims is statistically significant and not due to random chance. This finding implies that homicide cases involving White victims are solved at a significantly higher rate than those involving Black victims. Such a disparity could indicate systematic differences in investigative approaches or resource allocation—for example, it may reflect that cases involving White victims benefit from more available information or are subject to more thorough investigative follow-up. This result is directly relevant to our project objective, as it highlights potential racial biases in crime solving.

# Part 5. Modeling Data

In this modeling phase, our objective is to predict homicide case solvability—i.e., whether a case is solved (coded as 1) or unsolved (coded as 0). We start with Logistic Regression as a baseline due to its interpretability, and we then compare its performance with ensemble methods like XGBoost and Random Forest. Our models address class imbalance and are evaluated using accuracy, recall, precision, and F1-score to ensure robust performance on both training and testing data.

## 5.1 Data Preprocessing and Feature Engineering for Modeling

First, many of our key predictor variables—such as victim sex, victim race, weapon type, and agency type—were categorical in nature. To make these variables suitable for machine learning models, we applied One-Hot Encoding to create binary indicators for each category, setting drop_first=True to avoid multicollinearity issues.

Additionally, we dropped features that would not be available to law enforcement at the time of case intake, including perpetrator demographic characteristics and the perpetrator-victim relationship. Including these features introduces data leakage, as they would only be known after the case is resolved. Since our analysis focuses on predicting outcomes based on the information available during the investigation, we exclude these variables to avoid bias and ensure that the model reflects only the data law enforcement would have at the time.

Regarding geographic and socioeconomic factors, we initially considered including all U.S. Census-based state-level controls—such as population size, median household income, and high school attainment rates (sourced from the 2010 Census). However, these variables are time-invariant and may not capture important changes over our 1990–2014 sample period. Moreover, through experimentation, we found that incorporating detailed state fixed effects (via one-hot encoded State dummies) yielded higher model performance than using socioeconomic features alone. For this reason, we retained State indicators and excluded the Census-based socioeconomic variables from our final feature set.

We also removed variables that would obscure meaningful patterns due to excessive granularity (e.g., City, Date) or redundancy with already-included variables (e.g., Census Region, which is subsumed by State, and Firearm, which is fully captured by Weapon type).

Ultimately, we retain the following groups of features, largely based on the trends observed in our exploratory data analysis:

* Victim demographics: Sex, Race, Ethnicity, and Age are typically recorded at the time of the crime and provide important predictive signals.

* Incident characteristics: Month of the crime captures potential seasonal patterns, and weapon type provides information about the nature and severity of the homicide.

* Agency characteristics: Agency type (e.g., municipal police vs. state police) reflects differences in investigative capacity and clearance probabilities.

* Geographic context: State indicators account for jurisdictional differences in clearance practices, resources, and reporting standards.



In [ ]:
# Encode binary/categorical variables
df = murder_accountability_cleaned.copy()

# Drop columns that police would not have access to at time of crime/ features that overlap with others
cols_to_drop = ['Date', 'City', 'Relationship',  'Census_Region',
                'Perpetrator Sex', 'Perpetrator Race','Perpetrator Ethnicity', 'Perpetrator Age',
                'Relationship', 'Young', 'Firearm',
                'Population_2010', 'Income_2010', 'HS_Percent_2010', 'Year']

df.drop(columns=cols_to_drop, inplace=True)


# Convert the binary columns
binary_map = {'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0, 'True': 1, 'False': 0}
df['Crime Solved'] = df['Crime Solved'].map(binary_map)
df['Victim Sex'] = df['Victim Sex'].map(binary_map)

# cateogircal columns
categorical_to_encode = [
    'Agency Type',
    'Month',
    'Victim Race',
    'Victim Ethnicity',
    'Weapon',
    'State'
]

# drop_first=True to avoid multicollinearity from dummy variables.
df = pd.get_dummies(df, columns=categorical_to_encode, drop_first=True)
df_encoded = df.copy()

print("Final Encoded DataFrame Info:")
print(df_encoded.info())
print("\nFinal Encoded DataFrame Head:")
print(df_encoded.head())

We also create a second version of the dataset with more extensive feature engineering based on our EDA, to evaluate whether enhancing or reformatting certain variables could improve our model's predictive performance. Specifically, we bin the continuous victim age variable into 20-year intervals to better capture nonlinear relationships between age groups and clearance outcomes.

During our exploratory analysis, we observed that the state-level controls were moderately correlated with each other — particularly population size and education levels. To reduce the dimensionality and account for this correlation, we applied Principal Component Analysis (PCA) specifically to the set of state-level controls. After standardizing these three features, we extracted the first principal component, which captures approximately 50% of the total variance. This new variable, State_PC1, effectively summarizes overall state-level socioeconomic conditions and is included as a feature. We dropped the original three state-level variables to avoid redundancy and simplify model interpretation.

We also introduce a rolling 30-day local homicide count feature for each city, which serves as a proxy for police workload and investigative bandwidth—factors that may plausibly affect the likelihood of a case being solved. Cities with more homicides reported in the past month may be busier and have less resources to devote to each case.

Finally, we use an interaction term between the age and sex variables to capture non-linearities in the data. We saw in our EDA that crimes involving victims who are black males may have different clearance rates than white females. We explore this in our analysis.

First, to create the State_PC1 feature, we return to the original Census dataset and apply Principal Component Analysis (PCA) to the state-level socioeconomic variables. This technique captures the common variation across population, income, and education levels, allowing us to replace multiple correlated inputs with a single, compact measure. Because our exploratory data analysis (EDA) showed that these socioeconomic factors account for substantial geographic variation in homicide clearance rates, we drop "State" as a categorical variable and instead include State_PC1 to summarize these effects. Using one principal component avoids introducing 50 separate dummy variables for states, which could risk overfitting. Moreover, neighboring states often experience similar socioeconomic conditions and crime trends, which PCA can naturally capture more smoothly than treating each state as fully independent. (Note that since State_PC1 is computed solely from external Census data, it does not "peek" into our homicide dataset’s train/test split, preserving model integrity.)

In [ ]:
# Copy census state-level data
pca_df = census_data.copy().dropna()

# Select state-level controls
state_features = ['Population_2010', 'Income_2010', 'HS_Percent_2010']
state_data = pca_df[state_features]

# Standardize before PCA
scaler_state = StandardScaler()
state_scaled = scaler_state.fit_transform(state_data)

# Apply PCA
pca = PCA(n_components=1)
state_pc1 = pca.fit_transform(state_scaled)

# Add back PC1 as a new feature
pca_df['State_PC1'] = state_pc1

# Drop original state features
pca_df.drop(columns=state_features, inplace=True)

print("Explained Variance by State PC1:", pca.explained_variance_ratio_[0])
loadings = pd.Series(pca.components_[0], index=state_features)
print("\nPCA Loadings onto PC1:")
print(loadings)

All three features load positively onto PC1, with HS_Percent_2010 and Income_2010 exhibiting notably larger magnitudes compared to Population_2010.
This suggests that PC1 primarily captures a socioeconomic status and educational attainment dimension. States with higher educational attainment rates and higher median incomes receive higher PC1 scores. Thus, in subsequent analyses, the constructed State_PC1 variable can be interpreted as a general socioeconomic advantage index: higher values correspond to more educated, wealthier states, while lower values correspond to poorer, less educated states, independent of raw population size.


In [ ]:
df_features = murder_accountability_cleaned.copy()

# Merge PCA state-level variable
df_features = df_features.merge(
    pca_df[['State', 'State_PC1']],
    on='State',
    how='left'
)

# Create rolling average of prior number of monthly homicides in city
df_features['Date'] = pd.to_datetime(df_features['Date'])
df_features['YearMonth'] = df_features['Date'].dt.to_period('M')
city_month_counts = df_features.groupby(['City', 'YearMonth']).size().rename('Monthly_Homicides')
df_features = df_features.merge(city_month_counts, on=['City', 'YearMonth'], how='left')
df_features = df_features.sort_values(by='Date')

df_features['Prior_Month_Homicides'] = (
    df_features.groupby('City')['Monthly_Homicides'].shift(1).fillna(0)
)

# Drop helper columns
df_features.drop(columns=['Monthly_Homicides', 'YearMonth'], inplace=True)
df_features

In [ ]:
# bin victim ages
df_features['Victim_Age_Bin'] = pd.cut(
    df_features['Victim Age'],
    bins=range(0, 100, 20),
    right=False,
    include_lowest=True
).astype(str)


# Drop columns unavailable at case intake or redundant
cols_to_drop = [
    'Date', 'City', 'Relationship', 'Perpetrator Sex', 'Perpetrator Race',
    'Perpetrator Ethnicity', 'Perpetrator Age', 'Relationship',
    'Weapon', 'Population_2010', 'Income_2010', 'HS_Percent_2010',
    'State', 'Victim Age', 'Year'
]

df_features.drop(columns=cols_to_drop, inplace=True)

# Convert binary columns
binary_map = {'Yes': 1, 'No': 0, 'Male': 1, 'Female': 0}
df_features['Crime Solved'] = df_features['Crime Solved'].map(binary_map)
df_features['Victim Sex'] = df_features['Victim Sex'].map(binary_map)
df_features['Firearm'] = df_features['Firearm'].astype(int)
df_features['Young'] = df_features['Young'].astype(int)

# Interaction Term
df_features['Young_Male'] = df_features['Young'] * df_features['Victim Sex']
df_features['Black_Male'] = df_features['Victim Sex'] * (df_features['Victim Race'] == 'Black').astype(int)

categorical_to_encode = [
    'Agency Type',
    'Month',
    'Census_Region',
    'Victim Race',
    'Victim Ethnicity',
    'Victim_Age_Bin'
]

df_features = pd.get_dummies(df_features, columns=categorical_to_encode, drop_first=True)

# Final Check
print("\nFinal Feature Engineered DataFrame:")
print(df_features.info())
print(df_features.head())

## 5.2 Creating the Training and Testing Datasets

As we did in class, we decided to employ a 80 / 20 train-test-split for both our non-feature engineered and feature engineered datasets. Doing so ensures we have enough data to train the model while also being able to judge its performance on the test data. We set our seed = 42 as in our homeworks to ensure replicability.

In [ ]:
X = df_encoded.drop(columns=['Crime Solved'])
y = df_encoded['Crime Solved']

#train test split
SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y, shuffle=True)

'''
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
'''

print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')

In [ ]:
X_features = df_features.drop(columns=['Crime Solved'])
y_features = df_features['Crime Solved']

# train test split, f indicates feature engineering
SEED = 42
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_features, y_features, test_size=0.2, random_state=SEED, stratify=y_features, shuffle=True)

'''
X_train_f = X_train_f.to_numpy()
y_train_f = y_train_f.to_numpy()
X_test_f = X_test_f.to_numpy()
y_test_f = y_test_f.to_numpy()
'''

print(f'Training data shape: {X_train_f.shape}')
print(f'Testing data shape: {X_test_f.shape}')

In [ ]:
# Find the ratio
target_counts = y.value_counts()
majority_class_count = target_counts.max()
minority_class_count = target_counts.min()
class_ratio = majority_class_count / minority_class_count

print(f'Target variable has {class_ratio:.2f}x more observations in the negative (majority) class than the positive (minority) class.')

In our dataset, the target variable "Crime Solved" exhibits a moderate class imbalance, with the number of solved cases (the majority class) being approximately 2.27 times greater than the number of unsolved cases (the miority class). This imbalance is important to acknowledge because it can bias predictive models toward favoring the majority class (solved), leading to misleadingly high accuracy scores while failing to correctly predict the minority (unsolved) cases that are of primary interest. To address this issue and ensure balanced model performance, we explore model training strategies such as applying class_weight='balanced' in our Logistic Regression baseline. This adjustment re-weights observations inversely proportional to their class frequencies, allowing the model to treat solved and unsolved cases more equally during training.

Moreover, since the goal is to predict whether a homicide will be solved (binary classification), and the dataset exhibits moderate class imbalance, we evaluate the model using not only Accuracy, but also Precision, Recall, and F1-Score. Precision captures how often the model's positive predictions are correct, Recall captures the proportion of actual positive cases correctly identified, and F1-Score balances the trade-off between them. These metrics provide a more complete picture than Accuracy alone.

## 5.3 Logistic Regression

We begin our modeling analysis with Logistic Regression because it offers a strong, interpretable baseline for binary classification problems like predicting homicide case solvability. Logistic Regression is particularly well-suited when the target variable is binary (solved vs. unsolved) and when we seek to understand the magnitude and direction of association between features and the probability of case resolution. Its coefficients can be easily interpreted as odds ratios, providing direct insights into how factors such as victim characteristics, weapon type, or agency type influence clearance likelihood. By first fitting a Logistic model, we establish a benchmark against which to compare the performance gains of more complex machine learning models like Random Forests and XGBoost.

However, there are limitations of Logistic Regression models in this setting. For one, Logistic Regression assumes a linear relationship between predictors and the log-odds of the outcome. If relationships are highly nonlinear (e.g., interactions between race and agency type), Logistic Regression may underperform. Also, although we carefully one-hot encode variables and drop redundant columns (using drop_first=True), the presence of correlated predictors (e.g., race and weapon type) could inflate standard errors. Finally, although the class imbalance is moderate (2.27:1), Logistic Regression without class weighting could still favor the majority class. To address this, we implement a version of the model using class_weight='balanced' to ensure that unsolved and solved cases are treated appropriately during training.

While Principal Component Analysis (PCA) is a powerful technique for reducing dimensionality and addressing multicollinearity, we ultimately decided not to apply PCA to our main feature set for several reasons:


*   Loss of Interpretability:
PCA transforms original variables into linear combinations ("principal components"), which obscures the relationship between individual predictors and the outcome. Since one of our core goals is actionable insight for law enforcement, interpretability is critical and PCA might compromise this goal.


*   No Overwhelming Need for Dimensionality Reduction:
Our feature set is not so large as to pose risks of overfitting or computation
al inefficiency that would typically necessitate PCA.

* Minimal Multicollinearity Observed: After performing a correlation analysis on the core MAP variables, we did not observe strong multicollinearity that would necessitate dimensionality reduction. Most pairwise correlations were moderate to low in the core MAP dataset.




### 5.3.1 Building the Logistic Regression Model

Prior to model fitting, we standardized all continuous numerical variables (here, Victim Age) to have zero mean and unit variance using StandardScaler. Some models might give a feature that is larger in scale more influence in the prediction, which scaling helps mitigate. Binary dummy variables (e.g., Victim Race, Weapon Type, Agency Type, and State indicators) were left unscaled, preserving their original 0/1 values. This decision allows for clean and interpretable coefficient estimates for dummy features: the coefficient associated with a dummy variable reflects the change in log-odds of case clearance associated with the presence versus absence of that attribute (i.e., switching from 0 to 1).

In [ ]:
# Build plain Logistic Regression model and one using class_weight='balanced'
lrg = LogisticRegression(max_iter=1000, random_state=SEED)
weighted_lrg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED)

numerical_columns = ['Victim Age']  # only numerical continuous feature

# Copy the datasets
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Scale only the continuous numerical columns
scaler = StandardScaler()
X_train_scaled[numerical_columns] = scaler.fit_transform(X_train_scaled[numerical_columns])
X_test_scaled[numerical_columns] = scaler.transform(X_test_scaled[numerical_columns])

X_train_scaled = X_train_scaled.to_numpy()
X_test_scaled = X_test_scaled.to_numpy()

# Fit models
lrg.fit(X_train_scaled, y_train)
weighted_lrg.fit(X_train_scaled, y_train)

In [ ]:
# Predictions for plain Logistic Regression
lrg_y_pred_train = lrg.predict(X_train_scaled)
lrg_y_pred_test = lrg.predict(X_test_scaled)

# Predictions for weighted Logistic Regression
weighted_lrg_y_pred_train = weighted_lrg.predict(X_train_scaled)
weighted_lrg_y_pred_test = weighted_lrg.predict(X_test_scaled)

In [ ]:
# Evaluate performance - Plain LR
lrg_train_acc = accuracy_score(y_train, lrg_y_pred_train)
lrg_test_acc = accuracy_score(y_test, lrg_y_pred_test)
lrg_train_rec = recall_score(y_train, lrg_y_pred_train)
lrg_test_rec = recall_score(y_test, lrg_y_pred_test)
lrg_train_pre = precision_score(y_train, lrg_y_pred_train)
lrg_test_pre = precision_score(y_test, lrg_y_pred_test)
lrg_train_f1 = f1_score(y_train, lrg_y_pred_train)
lrg_test_f1 = f1_score(y_test, lrg_y_pred_test)

# Evaluate performance - Weighted LR
weighted_lrg_train_acc = accuracy_score(y_train, weighted_lrg_y_pred_train)
weighted_lrg_test_acc = accuracy_score(y_test, weighted_lrg_y_pred_test)
weighted_lrg_train_rec = recall_score(y_train, weighted_lrg_y_pred_train)
weighted_lrg_test_rec = recall_score(y_test, weighted_lrg_y_pred_test)
weighted_lrg_train_pre = precision_score(y_train, weighted_lrg_y_pred_train)
weighted_lrg_test_pre = precision_score(y_test, weighted_lrg_y_pred_test)
weighted_lrg_train_f1 = f1_score(y_train, weighted_lrg_y_pred_train)
weighted_lrg_test_f1 = f1_score(y_test, weighted_lrg_y_pred_test)

print('Regular Logistic Regression Performance')
print('---------------------------------------')
print(f'Training Accuracy: {lrg_train_acc*100:.2f}%')
print(f'Testing Accuracy: {lrg_test_acc*100:.2f}%')
print(f'Training Recall: {lrg_train_rec*100:.2f}%')
print(f'Testing Recall: {lrg_test_rec*100:.2f}%')
print(f'Training Precision: {lrg_train_pre*100:.2f}%')
print(f'Testing Precision: {lrg_test_pre*100:.2f}%')
print(f'Training F1-Score: {lrg_train_f1*100:.2f}%')
print(f'Testing F1-Score: {lrg_test_f1*100:.2f}%')

print('\nClass Weighted Logistic Regression Performance')
print('----------------------------------------------')
print(f'Training Accuracy: {weighted_lrg_train_acc*100:.2f}%')
print(f'Testing Accuracy: {weighted_lrg_test_acc*100:.2f}%')
print(f'Training Recall: {weighted_lrg_train_rec*100:.2f}%')
print(f'Testing Recall: {weighted_lrg_test_rec*100:.2f}%')
print(f'Training Precision: {weighted_lrg_train_pre*100:.2f}%')
print(f'Testing Precision: {weighted_lrg_test_pre*100:.2f}%')
print(f'Training F1-Score: {weighted_lrg_train_f1*100:.2f}%')
print(f'Testing F1-Score: {weighted_lrg_test_f1*100:.2f}%')


In [ ]:
# Confusion matrices
conf_matrix_plain = confusion_matrix(y_test, lrg.predict(X_test_scaled))
conf_matrix_weighted = confusion_matrix(y_test, weighted_lrg.predict(X_test_scaled))
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plain Logistic Regression
disp_plain = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_plain,
                                    display_labels=["Unsolved (0)", "Solved (1)"])
disp_plain.plot(cmap='Blues', values_format='d', ax=axes[0], colorbar=False)
axes[0].set_title('Plain Logistic Regression')

# Weighted Logistic Regression
disp_weighted = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_weighted,
                                       display_labels=["Unsolved (0)", "Solved (1)"])
disp_weighted.plot(cmap='Oranges', values_format='d', ax=axes[1], colorbar=False)
axes[1].set_title('Weighted Logistic Regression')
plt.tight_layout()
plt.show()

### 5.3.2 Performance Analysis

1.   **Overall performance**
We first evaluate the performance of our plain Logistic Regression and class-weighted Logistic Regression models. The plain Logistic Regression model achieves a testing accuracy of 71.1% and a testing F1-score of 81.7%, with very strong recall (93.0%) and decent precision (72.9%). This indicates that the model is highly effective at identifying cases that are eventually solved, though it also produces a moderate number of false positives. In contrast, the class-weighted Logistic Regression achieves a lower testing accuracy of 64.5% and F1-score of 71.0%. While it improves precision slightly to 81.6%, it suffers from a significantly lower recall (62.7%), meaning it misses a large portion of true solved cases.

When examining the confusion matrices, we see that the plain Logistic Regression model correctly identifies 52,176 solved cases but also incorrectly classifies 19,430 unsolved cases as solved, resulting in a high number of false positives. The weighted Logistic Regression reduces false positives to 7,784, but at the cost of identifying only 35,167 solved cases, missing nearly 21,000 solvable homicides. The weighted model is more conservative but risks failing to flag many solvable cases, which could be problematic when prioritizing law enforcement resources.

Despite the slightly higher precision achieved by the class-weighted logistic regression model, we ultimately favor the regular logistic regression model for our objectives. The regular logistic regression achieves significantly higher recall on the testing set (93% vs 63%), ensuring that the vast majority of solvable cases are successfully identified. Although this comes at the cost of some precision, the severe loss of recall in the class-weighted model undermines its practical utility. Thus, we conclude that the regular logistic regression model offers a better overall balance of recall, precision, and F1-Score, and is more appropriate for identifying cases at risk of remaining unsolved.


Areas for Improvement:
1. Boost Recall and Balance Precision:
The weighted Logistic Regression model struggles with recall (~63%). Future improvements could involve adjusting the classification threshold or using resampling techniques like SMOTE to better capture solvable cases.

2. Explore Nonlinear Models:
Logistic Regression assumes linear effects. Using models like Random Forests or XGBoost could uncover more complex patterns between victim, weapon, and agency characteristics.

3. Enhance Feature Engineering and Tuning:
Creating interaction terms (e.g., between weapon and victim demographics) and fine-tuning model hyperparameters could further improve predictive performance.

### 5.3.3 Feature Importance

In [ ]:
# Interpret Coefficients
coefficients = lrg.coef_[0]
features = X.columns
coef_df = pd.DataFrame({'Feature': features,
                        'Coefficient': coefficients,
                        'Odds Ratio': np.exp(coefficients)})

# Sort by absolute value of coefficient for interpretation
coef_df['Abs Coefficient'] = coef_df['Coefficient'].abs()
coef_top15 = coef_df.sort_values('Abs Coefficient', ascending=False).head(15)
print("\nTop 15 Features by Coefficient Magnitude:")
print(coef_top15[['Feature', 'Coefficient', 'Odds Ratio']])

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=top_coef_df, y='Feature', x='Odds Ratio', palette='viridis')
plt.title('Top 15 Features by Odds Ratio Magnitude: Baseline Logistic', fontsize=16)
plt.xlabel('Odds Ratio', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.axvline(x=1, color='red', linestyle='--')
plt.tight_layout()
plt.show()

**Some Observations from Logistic Regression Model**

The logistic‐regression baseline—trained only on victim and situational features available at case intake—achieves an F1 score of 0.817, demonstrating solid predictive power for flagging cases at risk of going cold. In logistic regression, coefficients represent the change in log-odds of the dependent variable — in this case, the likelihood that a homicide case is solved — associated with a one-unit change in the predictor, holding all other variables constant.

Since the features shown here are binary indicators (e.g., state, agency type, weapon type), each coefficient reflects the change in log-odds of clearance when the attribute is present (1) versus absent (0).

Key takeaways:

* State-level indicators absorb a large share of the variation in clearance rates. Cases in certain smaller or less densely populated states, such as Idaho, South Carolina, Montana, and others, are associated with substantially higher log-odds of being solved relative to the omitted baseline state. Conversely, cases in major urban areas like New York and the District of Columbia exhibit significantly lower log-odds of clearance.

* Cases handled by a State Police agency more likely to be solved as compared to cases handled by other types of law enforcement agencies.

* The use of a firearm as the weapon in the homicide decreases the chances of clearance compared to homicides involving other weapons, consistent with the challenges firearm-related crimes pose for investigators (i.e. less fingerprint evidence, easier to fire from a distance, etc.).

**Implications for practice**

These results suggest that geographic location plays a dominant role in clearance outcomes, but characteristics of the weapon used and the responding agency also significantly influence case solvability. It could be the case that police departments in populated states are more burdened with crime, including non-homicide crimes, and thus less able to investigate homicides, leading to lower clearance rates. There could also be some non-linearities/ interaction terms with the demographic composition of the state-- urban centers tend to be more diverse. These differences might be getting captured in the state dummies, preventing victim-level characteristics like race or sex from appearing significant.

### 5.3.4 Feature Engineering & Logistic Regression Model Performance

While our plain Logistic Regression model demonstrates strong baseline performance—achieving a testing F1-score of 81.7% and a recall rate above 93%—there remain clear opportunities for improvement. Specifically, the model shows a moderate rate of false positives, and the weighted Logistic Regression struggles with recall, suggesting that more sophisticated modeling approaches could enhance predictive accuracy. To address these limitations, we next explore whether enhanced feature engineering can further boost performance. In particular, we use the feature-engineered dataset we constructed that incorporates additional interaction terms, a principal component summarizing state-level socioeconomic characteristics, binned victim age groups, and contextual information such as recent homicide counts in the city.

We repeat the steps above using our feature-engineered dataset to see if we gain any predictive power. Here, State PC1 is the only continous numerical variable as we binned victim age, so we add it to the standardization. Even though PCA outputs components already centered at mean 0, the standard deviation may not be 1, so we scale this variable again across all crimes (some states may be more represented in our sample.)

In [ ]:
# Build plain Logistic Regression model for feature-engineered data
lrg = LogisticRegression(max_iter=1000, random_state=SEED)
numerical_columns = ["State_PC1"]  # only numerical continuous feature since victim age is binned

# Scale and Fit models on training data
scaler = StandardScaler()

# Copy the datasets
X_train_scaled_f = X_train_f.copy()
X_test_scaled_f = X_test_f.copy()

# Scale only the continuous columns
scaler = StandardScaler()
X_train_scaled_f[numerical_columns] = scaler.fit_transform(X_train_scaled_f[numerical_columns])
X_test_scaled_f[numerical_columns] = scaler.transform(X_test_scaled_f[numerical_columns])

lrg.fit(X_train_scaled_f, y_train_f)

In [ ]:
# Predictions for plain Logistic Regression, Feature Engineered Data
lrg_y_pred_train_f = lrg.predict(X_train_scaled_f)
lrg_y_pred_test_f = lrg.predict(X_test_scaled_f)

In [ ]:
# Evaluate performance - Plain LR, Feature Engineered Data
lrg_train_acc = accuracy_score(y_train_f, lrg_y_pred_train_f)
lrg_test_acc = accuracy_score(y_test_f, lrg_y_pred_test_f)
lrg_train_rec = recall_score(y_train_f, lrg_y_pred_train_f)
lrg_test_rec = recall_score(y_test_f, lrg_y_pred_test_f)
lrg_train_pre = precision_score(y_train_f, lrg_y_pred_train_f)
lrg_test_pre = precision_score(y_test_f, lrg_y_pred_test_f)
lrg_train_f1 = f1_score(y_train_f, lrg_y_pred_train_f)
lrg_test_f1 = f1_score(y_test_f, lrg_y_pred_test_f)


print('Regular Logistic Regression Performance: Feature Engineered Data')
print('---------------------------------------')
print(f'Training Accuracy: {lrg_train_acc*100:.2f}%')
print(f'Testing Accuracy: {lrg_test_acc*100:.2f}%')
print(f'Training Recall: {lrg_train_rec*100:.2f}%')
print(f'Testing Recall: {lrg_test_rec*100:.2f}%')
print(f'Training Precision: {lrg_train_pre*100:.2f}%')
print(f'Testing Precision: {lrg_test_pre*100:.2f}%')
print(f'Training F1-Score: {lrg_train_f1*100:.2f}%')
print(f'Testing F1-Score: {lrg_test_f1*100:.2f}%')

When comparing the feature-engineered logistic regression model to the original baseline model, we observe that overall performance remains relatively stable. The feature-engineered model achieves a testing F1-score of 81.6%, nearly identical to the baseline model's 81.7%. Testing recall is slightly higher in the feature-engineered model (94.6% vs. 93.0%), meaning it is even better at capturing solvable cases. However, this comes with a small tradeoff in testing precision (71.71% compared to 72.87%), indicating a slight increase in false positives. Despite adding interaction terms, PCA-transformed socioeconomic information, binned age groups, and local crime intensity measures, the feature enhancements did not substantially improve overall model performance.

We check to see if the most important features are the similar across the two datasets.

In [ ]:
# Interpret Coefficients
coefficients = lrg.coef_[0]
features = X_features.columns
coef_df = pd.DataFrame({'Feature': features,
                        'Coefficient': coefficients,
                        'Odds Ratio': np.exp(coefficients)})

# Sort by absolute value of coefficient for interpretation
coef_df['Abs Coefficient'] = coef_df['Coefficient'].abs()
coef_top15 = coef_df.sort_values('Abs Coefficient', ascending=False).head(15)
print("\nTop 15 Features by Coefficient Magnitude:")
print(coef_top15[['Feature', 'Coefficient', 'Odds Ratio']])

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=coef_top15, y='Feature', x='Odds Ratio', palette='viridis')
plt.title('Top 15 Features by Odds Ratio Magnitude: Feature Engineering Logistic', fontsize=16)
plt.xlabel('Odds Ratio', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.axvline(x=1, color='red', linestyle='--')
plt.tight_layout()
plt.show()

These results reveal an interesting pattern.

When using detailed state fixed effects (i.e., including a separate dummy variable for each state), the logistic regression results were dominated by geographic location. Nearly all of the top features by coefficient magnitude were individual state indicators, with states such as Idaho, South Carolina, and Montana associated with significantly higher odds of homicide case clearance, while states like New York and the District of Columbia were associated with lower odds. This pattern highlights substantial variation in clearance rates across states, likely reflecting differences in law enforcement practices, case management resources, or administrative reporting standards. However, the dominance of state dummies in the model absorbed most of the variation, limiting the ability to detect the influence of victim demographics or crime-specific characteristics.

When we replaced individual state fixed effects with broader controls — namely, Census Region indicators and a principal component summarizing state-level socioeconomic conditions (State_PC1) — the structure of feature importance shifted markedly. Victim-level and crime-specific attributes emerged as the primary drivers of variation in case clearance. In particular, victim age groups (especially victims aged 80 and older), agency type (State Police/ Sheriff agencies), the use of firearms, and victim characteristics such as sex, race, and ethnicity showed strong associations with solvability. For example, homicides involving firearms or Black victims were associated with lower log-odds of clearance, while cases handled by State Police agencies or involving non-Hispanic victims showed higher odds of being solved. This transition suggests that after accounting for broader regional differences, the model is better able to detect how individual case characteristics — rather than geographic assignment alone — affect the likelihood of resolution.

Key Takeaways
*  Victim Age: Homicides involving older victims, particularly those aged 80 and above, are much less likely to be solved.
*   Victim Race and Sex: Cases involving Black victims and male victims have lower odds of clearance, reflecting persistent disparities in investigative outcomes. On the other hand, cases where the victim is white or non-hispanic are much more likely to be solved.
*   Weapon Type: The use of a firearm significantly decreases the likelihood of a case being solved, perhaps due to fewer eyewitnesses and limited physical evidence.
*   Agency Type: Cases investigated by State Police or Sheriff’s offices show higher clearance odds, suggesting the importance of agency resources and jurisdiction.
*   Geography: Cases occurring in the Census South and West regions are somewhat more likely to be cleared, while higher State_PC1 values (indicating higher socioeconomic conditions) are associated with lower clearance probabilities. One potential reason is that in more urbanized, high-income states, police forces may be busier, larger caseloads. Further, there may be higher crime complexity in wealthier, more populous states.

Overall, the move from fine-grained state controls to broader regional controls enabled a clearer understanding of the substantive factors influencing homicide clearance. While geography remains important, particularly through the Census Region and State_PC1 variables, the model's predictive power now more clearly reflects attributes intrinsic to the crime and the victim rather than the location alone. This model achieves similar predictive power, suggesting that fine-grained information about the victim's characteristics is also important in terms of predicting power, without requiring exact state-level characteristics.

Because the model trained on the feature-engineered dataset performs similarly to the original model with a slightly lower F1-score, we proceed with the simpler, original set of predictors for our main subsequent analyses. Retaining the original feature set improves model interpretability, which is critical for understanding how individual case characteristics influence clearance outcomes. In particular, the principal component summarizing state socioeconomic conditions, while statistically valid, is less transparent than the original state dummies, which allow us to identify specific states associated with higher or lower clearance probabilities. By maintaining a direct link between model inputs and real-world attributes, we ensure that the results remain actionable and interpretable for law enforcement agencies. Further, we believe there is value in capturing state-specific differences in clearance rates, as the state dummies tell us which state agencies should prioritize resources to solving homicides more, and which one may not need to increase expenditure towards this.



### 5.3.5 Hyperparameter Tuning the Logistic Regression Model

While logistic regression is relatively simple compared to more complex machine learning models, it still has important hyperparameters that can significantly affect model performance, particularly in cases of class imbalance or multicollinearity among features. In our baseline model, we used default settings, but here we systematically tune key hyperparameters using cross-validation to optimize predictive performance.

We focus on tuning the following:
* Penalty (penalty): Type of regularization (e.g., L1, L2).

* Inverse Regularization Strength (C): Controls how much we penalize large coefficients. Smaller values imply stronger regularization.

Rather than manually adjusting parameters one at a time, we adopt a more systematic approach using Randomized Search Cross-Validation (RandomizedSearchCV). This method allows us to efficiently explore a broad range of hyperparameter values and identify a model configuration that better optimizes predictive performance. RandomizedSearchCV is less exhaustive than GridSearchCV because it does not evaluate every combination of hyperparameters, but given the size of our dataset (300K + rows), is more suitable for optimization. The evaluation metric we optimize is the F1-score, which balances precision and recall — particularly important given the class imbalance in homicide clearance data.

In [ ]:
lrg = LogisticRegression(max_iter=1000, random_state=SEED)

# parameter grid
param_dist = {
    'penalty': ['l1', 'l2'], # regularization options
    'C': np.logspace(-2, 2, 20),  # test values between 1e-2 and 1e2
    'solver': ['liblinear'] # initially tried tuning this too, but took too long to run
}

# Setup RandomizedSearchCV
random_search_lrg = RandomizedSearchCV(
    estimator=lrg,
    param_distributions=param_dist,
    n_iter=10,  # number of random parameter combinations to try
    scoring='f1',  # optimize for F1-score because of class imbalnce
    cv=3,
    verbose=3,
    random_state=SEED,
    n_jobs=-1
)

random_search_lrg.fit(X_train, y_train)
best_lrg = random_search_lrg.best_estimator_

print("Best Hyperparameters:", random_search_lrg.best_params_)

# Evaluate tuned logistic regression
y_train_pred_lrg = best_lrg.predict(X_train)
y_test_pred_lrg = best_lrg.predict(X_test)

lrg_train_acc = accuracy_score(y_train_f, y_train_pred_lrg)
lrg_test_acc = accuracy_score(y_test_f, y_test_pred_lrg)
lrg_train_rec = recall_score(y_train_f, y_train_pred_lrg)
lrg_test_rec = recall_score(y_test_f, y_test_pred_lrg)
lrg_train_pre = precision_score(y_train_f, y_train_pred_lrg)
lrg_test_pre = precision_score(y_test_f, y_test_pred_lrg)
lrg_train_f1 = f1_score(y_train_f, y_train_pred_lrg)
lrg_test_f1 = f1_score(y_test_f, y_test_pred_lrg)


print('Hyperparameter Tuned Logistic Regression Performance')
print('---------------------------------------')
print(f'Training Accuracy: {lrg_train_acc*100:.2f}%')
print(f'Testing Accuracy: {lrg_test_acc*100:.2f}%')
print(f'Training Recall: {lrg_train_rec*100:.2f}%')
print(f'Testing Recall: {lrg_test_rec*100:.2f}%')
print(f'Training Precision: {lrg_train_pre*100:.2f}%')
print(f'Testing Precision: {lrg_test_pre*100:.2f}%')
print(f'Training F1-Score: {lrg_train_f1*100:.2f}%')
print(f'Testing F1-Score: {lrg_test_f1*100:.2f}%')

After hyperparameter tuning, the Logistic Regression model achieved a testing F1-Score of 81.77%, slightly higher than the 81.73% for the baseline model. Given the minimal gains in predictive performance, we conclude that hyperparameter tuning does not significantly enhance the logistic regression model's ability to predict homicide clearance outcomes in this setting. The model's performance appears relatively robust to changes in regularization strength and penalty choice. Since the logistic regression model has fewer hyperparameters to tune than more advanced models like XGBoost and Random Forest, it will be interesting to see if hyperparameter tuning is more effective for those models than the marginal benefit we see here.

## 5.4 XGBoost

While logistic regression provides a highly interpretable baseline and performs reasonably well, it is inherently limited by its linearity assumption — that each feature contributes additively and independently to the log-odds of case clearance. This may fail to capture complex, nonlinear interactions between victim characteristics, crime circumstances, and agency factors. To address these shortcomings, we next turn to XGBoost, a powerful ensemble method capable of modeling nonlinear relationships and higher-order feature interactions. XGBoost also provides feature importance measures, helping us better identify which factors most strongly influence solvability while potentially improving overall predictive accuracy. Additionally, given the class imbalance in the dataset (more solved than unsolved cases), we employ the scale_pos_weight parameter in XGBoost to properly reweight positive classes.  Because XGBoost is built on decision trees, which are inherently scale-invariant, we chose not to apply PCA-transformed or standardized features to the training and testing data. Instead we use our original data (not relying on the StandardScalar).


However, XGBoost has some limitations. First, it operates as a more "black-box" model, making it considerably harder to interpret than logistic regression. While coefficients in the logistic regression model can easily be interpreted as a change in the odds of a case being solved based on observable characteristics, features do not have as easy an interpretation with XGBoost. This is a tradeoff between predictive power and transparency that is particularly important in high-stakes settings like criminal justice. Namely, we might be interested in coefficient interpretation, which enables us to exactly compare different states and the chance in odds for Black and White victims. Further, it can be computationally intensive, especially when tuning hyperparameters or working with large datasets. Finally, XGBoost is prone to overfitting if regularization is not properly implemented or if the model is over-complex relative to the size and noise level of the training data. These considerations motivate model selection in practice, which we strive to do via hyper-paramater tuning.


### 5.4.1 Building a XGBoost Model

We choose hyperparameters that we believe will minimize overfitting while still capturing the complexity of the data. Namely, we set a max_depth of 3 to prevent overfitting given the number of features in our dataset, and choose 100 for the number of estimators. To account for class imbalances, we use set the scale_pos_weight = class ratio. These choices are somewhat arbitrary, but grounded in our desire to capture comlex patterns while preventing overfitting. We explore hyperparameter tuning in the next section.

In [ ]:
xgb_clf = XGBClassifier(random_state=SEED, tree_method='hist', scale_pos_weight=class_ratio, # scale_pos_weight accounts for class imbalances.
                        n_estimators=100, max_depth=3)

# Fit XGBoost model to the training data
xgb_clf.fit(X_train, y_train)

In [ ]:
xgb_clf_y_train_pred = xgb_clf.predict(X_train)
xgb_clf_y_test_pred = xgb_clf.predict(X_test)

# Evaluate Performance
xgb_train_acc = accuracy_score(y_train, xgb_clf_y_train_pred)
xgb_test_acc = accuracy_score(y_test, xgb_clf_y_test_pred)
xgb_train_rec = recall_score(y_train, xgb_clf_y_train_pred)
xgb_test_rec = recall_score(y_test, xgb_clf_y_test_pred)
xgb_train_pre = precision_score(y_train, xgb_clf_y_train_pred)
xgb_test_pre = precision_score(y_test, xgb_clf_y_test_pred)
xgb_train_f1 = f1_score(y_train, xgb_clf_y_train_pred)
xgb_test_f1 = f1_score(y_test, xgb_clf_y_test_pred)

print('\nXGBoost Classifier Performance:')
print(f'Training Accuracy: {xgb_train_acc*100:.2f}%')
print(f'Testing Accuracy: {xgb_test_acc*100:.2f}%')
print(f'Training Recall: {xgb_train_rec*100:.2f}%')
print(f'Testing Recall: {xgb_test_rec*100:.2f}%')
print(f'Training Precision: {xgb_train_pre*100:.2f}%')
print(f'Testing Precision: {xgb_test_pre*100:.2f}%')
print(f'Training F1-Score: {xgb_train_f1*100:.2f}%')
print(f'Testing F1-Score: {xgb_test_f1*100:.2f}%')


Although our baseline XGBoost model already achieves strong performance—particularly in terms of recall (99.71%) and F1-score (82.11%)—there is still room for optimization. Notably, while the model captures nearly all solvable cases, it does so at lower precision, suggesting that it may be over-predicting positive outcomes (i.e., labeling many unsolved cases as solved). To further improve the balance between precision and recall, we proceed to hyperparameter tuning.

Using RandomizedSearchCV,, we tune important hyperparameters such as max_depth (tree depth) and n_estimators (number of boosting rounds). By fine-tuning these hyperparameters, we aim to develop a model that not only maintains high recall but also improves precision, thereby boosting the overall F1-score.


### 5.4.2 Hyperparameter Tuned XGBoost

We adopt RandomizedSearchCV to efficiently explore a range of hyperparameters and prevent overfitting from manual tuning. The hyperparameters tuned include:

* n_estimators: number of boosting rounds,

* max_depth: maximum depth of trees,

* learning_rate: step size shrinkage,

* subsample: fraction of samples used per tree,

* colsample_bytree: fraction of features used per tree.

F1-Score is chosen as the optimization metric due to the dataset's moderate class imbalance and the need to balance precision and recall.

In [ ]:
#  parameter grid
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

# base XGBoost model
xgb_model = XGBClassifier(
    random_state=42,
    tree_method='hist',
    scale_pos_weight=class_ratio  # account for class imbalance
)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # number of random combinations to try
    scoring='f1',  # we select 'f1' for a balance between precision and recall
    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)
best_xgb_model = random_search.best_estimator_
print("Best Parameters found:", random_search.best_params_)


In [ ]:
xgb_best_clf_y_train_pred = best_xgb_model.predict(X_train)
xgb_best_clf_y_test_pred = best_xgb_model.predict(X_test)

xgb_best_train_acc = accuracy_score(y_train, xgb_best_clf_y_train_pred)
xgb_best_test_acc = accuracy_score(y_test, xgb_best_clf_y_test_pred)
xgb_best_train_rec = recall_score(y_train, xgb_best_clf_y_train_pred)
xgb_best_test_rec = recall_score(y_test, xgb_best_clf_y_test_pred)
xgb_best_train_pre = precision_score(y_train, xgb_best_clf_y_train_pred)
xgb_best_test_pre = precision_score(y_test, xgb_best_clf_y_test_pred)
xgb_best_train_f1 = f1_score(y_train, xgb_best_clf_y_train_pred)
xgb_best_test_f1 = f1_score(y_test, xgb_best_clf_y_test_pred)

print('\nHyperparameter Tuned XGBoost Classifier Performance:')
print(f'Training Accuracy: {xgb_best_train_acc*100:.2f}%')
print(f'Testing Accuracy: {xgb_best_test_acc*100:.2f}%')
print(f'Training Recall: {xgb_best_train_rec*100:.2f}%')
print(f'Testing Recall: {xgb_best_test_rec*100:.2f}%')
print(f'Training Precision: {xgb_best_train_pre*100:.2f}%')
print(f'Testing Precision: {xgb_best_test_pre*100:.2f}%')
print(f'Training F1-Score: {xgb_best_train_f1*100:.2f}%')
print(f'Testing F1-Score: {xgb_best_test_f1*100:.2f}%')

Hyperparameter tuning led to modest but meaningful improvements across several evaluation metrics. Testing Precision improved from 69.8% to 70.0%, suggesting that the model became slightly better at avoiding false positives. Testing F1-Score increased from 82.1% in the non-tuned model to 82.2%, indicating a slightly better overall balance between precision and recall. Testing Accuracy also rose slightly from 69.87% to 70.07%. Recall decreased slightly from 99.71% to 99.48%, but remains extremely high, meaning the model still captures nearly all solvable cases.

The small tradeoff in recall is acceptable, given the improvements in precision and F1-Score, which help reduce false positives without compromising the model’s core ability to identify solvable homicides. So, we use this hyper-paramater tuned XGBoost model going forward. However, note that hyperparameter tuning did not yield significant benefits in terms of the F-1 score compared to the XGBoost model we initially started with.

### 5.4.3 Performance Analysis

Comparing the hyperparameter-tuned models reveals distinct strengths between XGBoost and logistic regression. The tuned XGBoost model achieves a slightly lower Testing Accuracy (70.07%) compared to the tuned logistic regression (71.09%), reflecting logistic regression's relative advantage in overall classification accuracy. However, XGBoost delivers much higher Testing Recall (99.48% vs. 93.45%), meaning it identifies nearly all solvable cases, whereas logistic regression misses a larger fraction.

Testing Precision is somewhat higher for logistic regression (72.68%) than for XGBoost (70.00%), suggesting that logistic regression produces fewer false positives among cases predicted as solved. In terms of overall balance, XGBoost achieves a marginally higher Testing F1-Score (82.18% vs. 81.77%), indicating a slightly better tradeoff between precision and recall.

Overall, while tuned logistic regression provides better precision and slightly higher accuracy, the tuned XGBoost model's near-perfect recall and higher F1-Score make it better suited for tasks where identifying as many solvable cases as possible is a key priority.

### 5.4.4 Feature Importance

In [ ]:
best_xgb_model.get_booster().feature_names = list(X.columns)

# weight
xgb.plot_importance(
    best_xgb_model,
    importance_type='weight',
    max_num_features=10,
    height=0.8,
)
plt.title("Top 10 Features by Weight", fontsize=16)
plt.xlabel('Number of Splits', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.show()

# gain
xgb.plot_importance(
    best_xgb_model,
    importance_type='gain',
    max_num_features=10,
    height=0.8,
)
plt.title("Top 10 Features by Gain", fontsize=16)
plt.xlabel('Average Gain', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.show()

We analyze feature importance using two metrics: Weight (the number of times a feature is used to split data) and Gain (the improvement in model performance resulting from the feature). The top features identified by XGBoost based on Weight and Gain reveal both similarities and notable differences in how the model splits and relies on various predictors.

Looking first at crime-level and victim characteristics indicators dominate the top positions. Features such as Victim Age (2160 splits), Victim Sex (302 splits), and Weapon_Handgun (268 splits) (111 splits), State_New York (74 splits), and Agency Type_Sheriff (128 splits) are among the most frequently used features. This suggests that victim characteristics and weapon type are critical dimensions that the model repeatedly considers when partitioning the data.

However, examining Gain reveals a somewhat different emphasis. Features like State_South Carolina (Gain = 117.2), State_New York (Gain = 103.6), and State_California (Gain = 80.2) are prominent, as well as State_District of Columbia (Gain = 63.4) and State_Illinois (Gain = 62.3). These states, while not necessarily split on as frequently as others, contribute substantial information when they are used, significantly boosting model performance. Meanwhile, Weapon_Firearm shows relatively high Weight (frequent splits) but comparatively lower Gain (average Gain = 41.6), suggesting it is consistently informative but provides smaller incremental improvements each time it is used.

Overall, these findings align with our feature importance analysis for the logistic regression model, where we saw that states exhibit significant differences in their clearance rates and geographic variation is consistently an important determinant of whether a case will be solved or not.

### 5.4.5 Feature Engineering & XGBoost Model Performance

We briefly test whether using our feature engineered dataset boosts the predictive power of the XGBoost model. We use a smaller paramter grid to get the best XGBoost model since this feature engineered dataset is not our main one, but rather used to explore if feature engineering boosts model performance.

In [ ]:
X_train_f = X_train_f.to_numpy()
y_train_f = y_train_f.to_numpy()
X_test_f = X_test_f.to_numpy()
y_test_f = y_test_f.to_numpy()

#  parameter grid
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
}

# base XGBoost model
xgb_model = XGBClassifier(
    random_state=42,
    tree_method='hist',
    scale_pos_weight=class_ratio  # account for class imbalance
)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # number of random combinations to try
    scoring='f1',  # we select 'f1' for a balance between precision and recall

    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_f, y_train_f)
best_xgb_model_features = random_search.best_estimator_
print("Best Parameters found:", random_search.best_params_)

xgb_best_clf_y_train_pred = best_xgb_model_features.predict(X_train_f)
xgb_best_clf_y_test_pred = best_xgb_model_features.predict(X_test_f)

# evaluate performance
xgb_best_train_acc = accuracy_score(y_train_f, xgb_best_clf_y_train_pred)
xgb_best_test_acc = accuracy_score(y_test_f, xgb_best_clf_y_test_pred)
xgb_best_train_rec = recall_score(y_train_f, xgb_best_clf_y_train_pred)
xgb_best_test_rec = recall_score(y_test_f, xgb_best_clf_y_test_pred)
xgb_best_train_pre = precision_score(y_train_f, xgb_best_clf_y_train_pred)
xgb_best_test_pre = precision_score(y_test_f, xgb_best_clf_y_test_pred)
xgb_best_train_f1 = f1_score(y_train_f, xgb_best_clf_y_train_pred)
xgb_best_test_f1 = f1_score(y_test_f, xgb_best_clf_y_test_pred)

print('\nHyperparameter Tuned XGBoost Classifier Performance with Feature Engineering:')
print(f'Training Accuracy: {xgb_best_train_acc*100:.2f}%')
print(f'Testing Accuracy: {xgb_best_test_acc*100:.2f}%')
print(f'Training Recall: {xgb_best_train_rec*100:.2f}%')
print(f'Testing Recall: {xgb_best_test_rec*100:.2f}%')
print(f'Training Precision: {xgb_best_train_pre*100:.2f}%')
print(f'Testing Precision: {xgb_best_test_pre*100:.2f}%')
print(f'Training F1-Score: {xgb_best_train_f1*100:.2f}%')
print(f'Testing F1-Score: {xgb_best_test_f1*100:.2f}%')

Interestingly, we achieve a slightly higher F1-Score on the testing data when using our feature-engineered dataset compared to the non-feature engineered dataset and we see a slightly higher testing precision score. However, the imporvements are very incremental. Once again, this provides evidence that our feature-engineering did not significantly improve model performance. In future exploration, we could test with different interaction terms for feature-engineering to see if we can further improve model performance.

In [ ]:
best_xgb_model_features.get_booster().feature_names = list(X_features.columns)

# weight
xgb.plot_importance(
    best_xgb_model_features,
    importance_type='weight',
    max_num_features=10,
    height=0.8,
)
plt.title("Top 10 Features by Weight, Feature Engineered Data", fontsize=16)
plt.xlabel('Number of Splits', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.show()

# gain
xgb.plot_importance(
    best_xgb_model_features,
    importance_type='gain',
    max_num_features=10,
    height=0.8,
)
plt.title("Top 10 Features by Gain, Feature Engineered Data", fontsize=16)
plt.xlabel('Average Gain', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.show()

However, the differences in the feature importance are interesting to note. We can see that our engineered features, namely State_PC1 and Prior_Month_Homicides and Black_Male, play a large role in feature importance measured in terms of both weight and gain. This suggests that the model does use our engineered features in a way that improves its predictive power. We can see that victim characteristics are an important part of this model, as it does not just rely on state dummies like the one in 5.4.2. However, the feature with the highest gain is still based on geography (Census_Region_South), highlighting regional variation in homicides across different specifications. Black_Male is also significant, supporting our earlier finding of racial disparities in clearance rates.

## 5.5 Random Forest

In addition to XGBoost, we also implemented a Random Forest Classifier, another ensemble method based on decision trees. While XGBoost sequentially corrects errors through boosting, Random Forests aggregate predictions from independently grown trees using bagging. Including a Random Forest model allows us to assess whether the added complexity of boosting provides meaningful gains in predictive performance, particularly given the moderate class imbalance and nonlinear feature interactions present in the homicide clearance dataset.

That said, Random Forest models have some downsides. They can be slower when working with lots of features or deep trees. And like most ensemble models, they are not very easy to interpret — it is hard to explain exactly how a prediction was made, since it is averaged over so many trees. Still, they are a solid benchmark and give us a good point of comparison against other methods like neural networks. This method is similar to XGBoost and has many of the same pros and cons, so we expect similar results.



### 5.5.1 Building the Random Forest Model

First, we build a baseline Random Forest Model, setting the class weight to balanced to account for class imbalance. Once again, we choose parameters that we believe will minimize overfitting while capturing the complex patterns in our dataset. Namely, we choose a max depth of 5 (slightly larger than our non-tuned XGBoost model's max depth) and the same number of estimators of 100.

In [ ]:
# Initialize RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced', n_estimators=100,
                             max_depth=5, random_state=SEED)

rfc.fit(X_train, y_train)

# Random Forest Classifier Evaluation
rfc_y_train_pred = rfc.predict(X_train)
rfc_y_test_pred = rfc.predict(X_test)

rfc_train_acc = accuracy_score(y_train, rfc_y_train_pred)
rfc_test_acc = accuracy_score(y_test, rfc_y_test_pred)
rfc_train_rec = recall_score(y_train, rfc_y_train_pred)
rfc_test_rec = recall_score(y_test, rfc_y_test_pred)
rfc_train_pre = precision_score(y_train, rfc_y_train_pred)
rfc_test_pre = precision_score(y_test, rfc_y_test_pred)
rfc_train_f1 = f1_score(y_train, rfc_y_train_pred)
rfc_test_f1 = f1_score(y_test, rfc_y_test_pred)

print('\nRandom Forest Classifier Performance:')
print(f'Training Accuracy: {rfc_train_acc*100:.2f}%')
print(f'Testing Accuracy: {rfc_test_acc*100:.2f}%')
print(f'Training Recall: {rfc_train_rec*100:.2f}%')
print(f'Testing Recall: {rfc_test_rec*100:.2f}%')
print(f'Training Precision: {rfc_train_pre*100:.2f}%')
print(f'Testing Precision: {rfc_test_pre*100:.2f}%')
print(f'Training F1-Score: {rfc_train_f1*100:.2f}%')
print(f'Testing F1-Score: {rfc_test_f1*100:.2f}%')

The baseline Random Forest model performs substantially worse than the tuned XGBoost model, achieving a Testing F1-Score of only 69.07%, compared to 82.18% for XGBoost. The Random Forest also shows lower Testing Accuracy (62.53%) and Testing Recall (60.31%), indicating that it struggles both to correctly classify cases and to identify solvable homicides.

### 5.5.2 Hyperparameter Tuned Random Forest Model

We now hyperparameter tune the Random Forest model to see if we can improve on its predictive power. The parameters we tune are similar to the XGBoost model. Given the class imbalance, we continue to optimize over the F-1 score which balances precision and recall.

In [ ]:
# Define the stratified cross-validation splitter
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

# initialize a RandomForestClassifier with only random_state set
estimator = RandomForestClassifier(random_state=SEED)

# Set up the parameter grid to tune class_weight, n_estimators, and max_depth
param_grid = {
    'class_weight': [None, 'balanced'], # account for class imbalance in model or not
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10] # max depth of trees, should capture complexities without overfitting
}

# Choose a metric to optimize; here we select 'f1' for a balance between precision and recall
scoring = 'f1'

# Set up the grid search
search = GridSearchCV(estimator, param_grid, cv=cv, scoring=scoring, verbose=2)
search.fit(X_train, y_train)

# Evaluate the best model on the test set using the selected metric
search_score = search.score(X_test, y_test)
print('\nBest hyperparameters found:')
print(search.best_params_)
print(f'Score on the test set with the best model: {search_score:.4f}')

In [ ]:
# Get the best Random Forest model
best_rfc = search.best_estimator_

# Predict on train and test sets
rfc_best_train_pred = best_rfc.predict(X_train)
rfc_best_test_pred = best_rfc.predict(X_test)

# Evaluate performance
rfc_best_train_acc = accuracy_score(y_train, rfc_best_train_pred)
rfc_best_test_acc = accuracy_score(y_test, rfc_best_test_pred)
rfc_best_train_rec = recall_score(y_train, rfc_best_train_pred)
rfc_best_test_rec = recall_score(y_test, rfc_best_test_pred)
rfc_best_train_pre = precision_score(y_train, rfc_best_train_pred)
rfc_best_test_pre = precision_score(y_test, rfc_best_test_pred)
rfc_best_train_f1 = f1_score(y_train, rfc_best_train_pred)
rfc_best_test_f1 = f1_score(y_test, rfc_best_test_pred)

# Print performance
print('\nTuned Random Forest Classifier Performance:')
print(f'Training Accuracy: {rfc_best_train_acc*100:.2f}%')
print(f'Testing Accuracy: {rfc_best_test_acc*100:.2f}%')
print(f'Training Recall: {rfc_best_train_rec*100:.2f}%')
print(f'Testing Recall: {rfc_best_test_rec*100:.2f}%')
print(f'Training Precision: {rfc_best_train_pre*100:.2f}%')
print(f'Testing Precision: {rfc_best_test_pre*100:.2f}%')
print(f'Training F1-Score: {rfc_best_train_f1*100:.2f}%')
print(f'Testing F1-Score: {rfc_best_test_f1*100:.2f}%')


After hyperparameter tuning, the Random Forest model exhibits a dramatic improvement in predictive performance. The Tuned Random Forest achieves a Testing F1-Score of 82.21%, slightly exceeding the tuned XGBoost model. Testing Accuracy also improves significantly to 70.30%, and Testing Recall rises sharply to 98.92%, while maintaining Testing Precision at 70.33%. Unlike the more modest gains observed for XGBoost and logistic regression, Random Forest benefits substantially from hyperparameter tuning, particularly through better depth control and class-weight adjustment. Overall, these results suggest that although Random Forest initially underperforms relative to XGBoost, careful tuning can elevate its performance to a comparable level. This highlights the importance of hyperparameter tuning in achieving optimal predictive accuracy, especially for ensemble methods like Random Forest that rely heavily on parameter configuration.

### 5.5.3 Performance Analysis

Comparing the three tuned models reveals important trade-offs between overall accuracy, recall, precision, and model complexity.

Logistic Regression achieves the highest Testing Accuracy (71.1%) and the highest Testing Precision (72.7%), suggesting it is best at minimizing false positives and maintaining clean classification boundaries. However, its Testing Recall is substantially lower (93.45%) than the ensemble models, indicating that logistic regression misses a meaningful share of solvable cases. The Testing F1-Score (81.77%) reflects this imbalance, despite strong precision.

XGBoost outperforms logistic regression in terms of recall (99.48%) and F1-Score (82.18%), achieving a better balance between correctly identifying solvable cases and maintaining reasonable precision (70.00%). Although XGBoost's Testing Accuracy (70.07%) is slightly lower than logistic regression’s, its extremely high recall makes it more reliable for detecting positive outcomes, which is critical in the context of prioritizing solvable homicides.

Random Forest, after hyperparameter tuning, achieves performance nearly identical to XGBoost. Its Testing Accuracy (70.30%) slightly exceeds that of XGBoost, and its Testing F1-Score (82.21%) matches or slightly edges out XGBoost's. Testing Recall (98.92%) remains extremely high, although marginally lower than XGBoost, while Testing Precision (70.33%) is slightly higher. Given that the baseline Random Forest model performed much worse before tuning, these results underscore the sensitivity of Random Forest performance to careful hyperparameter optimization.

In sum, logistic regression excels in overall accuracy and precision but at the cost of lower recall, making it less desirable when the primary objective is to capture as many solvable cases as possible. XGBoost and Random Forest both achieve superior recall and slightly higher F1-scores, with Random Forest narrowly matching XGBoost after tuning. However, these models were all similar, suggesting that the features do contrinute in relatively linear, non-complex ways to homicide clearance as more complex models than logistic regression did not drastically improve any metric.

### 5.5.4 Feature Importance

Scikit-learn's Random Forest Feature Importance uses Gini impurity to assess the contribution of each feature in the model’s prediction. During training, Random Forest builds multiple decision trees by randomly selecting features at each split. The Gini impurity is calculated for each feature split, which measures how often a feature is used to reduce impurity (uncertainty) in the dataset. Features that lead to greater reductions in Gini impurity are deemed more important. This allows us to identify which features have the most influence on predicting homicide clearrance.

In [ ]:
importances = best_rfc.feature_importances_

# plot top 10 important features for rf model
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

feature_importance_df['Importance'] = feature_importance_df['Importance'].round(3)

top_n = 10
top_features = feature_importance_df.head(top_n)

plt.figure(figsize=(8, 6))
plt.barh(top_features['Feature'], top_features['Importance'])
plt.xlabel('Feature Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.title(f'Top {top_n} Feature Importances (Random Forest)', fontsize=16)
plt.gca().invert_yaxis()
plt.show()

Comparing the feature importance rankings between Random Forest and XGBoost reveals notable similarities as well as key differences in how the two ensemble methods prioritize predictors.

Both models identify Weapon_Firearm, State_New York, State_California, and Agency Type_Sheriff as highly influential features. This consistency suggests that both algorithms recognize weapon type, state-level context, and law enforcement agency characteristics as central drivers of homicide case solvability.

However, the models differ in emphasis. In Random Forest, victim demographics — including Victim Sex, Victim Age, and Victim Race (Black and White) — appear prominently among the top features. In contrast, XGBoost assigns much greater importance to state indicators, particularly southern and midwestern states (e.g., South Carolina, Tennessee, Illinois), with victim-specific variables appearing less prominently. This difference could reflect the nature of the algorithms: Random Forest, which builds trees independently, tends to distribute importance more evenly across victim-level and crime-specific features. XGBoost, which sequentially corrects errors, identifies specific geographic splits as yielding the largest performance gains, resulting in greater importance assigned to particular states.

Weapon type importance is somewhat consistent: Weapon_Firearm is among the top features for both models, though it carries relatively higher relative weight in Random Forest than in XGBoost based on Gain. This suggests that while both models recognize firearms as critical, Random Forest relies more heavily and uniformly on this feature across splits. Overall, XGBoost appears more sensitive to geographic heterogeneity, while Random Forest surfaces a broader mix of victim demographics, weapon types, and agency characteristics. These differences highlight how ensemble methods, even when achieving similar predictive performance, can vary in the underlying structure of their decision-making processes.


### 5.5.6 Feature Engineering & Random Forest Model Performance

We also test the Random Forest model on the feature engineered dataset, using a smaller parameter grid as this is not our main analysis.

In [ ]:
# Define the stratified cross-validation splitter
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

# initialize a RandomForestClassifier with only random_state set
estimator = RandomForestClassifier(random_state=SEED)

# Set up the parameter grid to tune class_weight, n_estimators, and max_depth
param_grid = {
    'class_weight': [None, 'balanced'],
    'n_estimators': [100, 150],
    'max_depth': [5, 10]
}

# Choose a metric to optimize; here we select 'f1' for a balance between precision and recall
scoring = 'f1'

# Set up the grid search
search_f = GridSearchCV(estimator, param_grid, cv=cv, scoring=scoring, verbose=2)
search_f.fit(X_train_f, y_train_f)

# Evaluate the best model on the test set using the selected metric
search_score = search_f.score(X_test_f, y_test_f)
print('\nBest hyperparameters found:')
print(search.best_params_)
print(f'Score on the test set with the best model: {search_score:.4f}')

In [ ]:
# Get the best Random Forest model
best_rfc_f = search_f.best_estimator_

# Predict on train and test sets
rfc_best_train_pred = best_rfc_f.predict(X_train_f)
rfc_best_test_pred = best_rfc_f.predict(X_test_f)

# Evaluate performance
rfc_best_train_acc = accuracy_score(y_train, rfc_best_train_pred)
rfc_best_test_acc = accuracy_score(y_test, rfc_best_test_pred)
rfc_best_train_rec = recall_score(y_train, rfc_best_train_pred)
rfc_best_test_rec = recall_score(y_test, rfc_best_test_pred)
rfc_best_train_pre = precision_score(y_train, rfc_best_train_pred)
rfc_best_test_pre = precision_score(y_test, rfc_best_test_pred)
rfc_best_train_f1 = f1_score(y_train, rfc_best_train_pred)
rfc_best_test_f1 = f1_score(y_test, rfc_best_test_pred)

# Print performance
print('\nTuned Random Forest Classifier Performance: Feature Engineered Data')
print(f'Training Accuracy: {rfc_best_train_acc*100:.2f}%')
print(f'Testing Accuracy: {rfc_best_test_acc*100:.2f}%')
print(f'Training Recall: {rfc_best_train_rec*100:.2f}%')
print(f'Testing Recall: {rfc_best_test_rec*100:.2f}%')
print(f'Training Precision: {rfc_best_train_pre*100:.2f}%')
print(f'Testing Precision: {rfc_best_test_pre*100:.2f}%')
print(f'Training F1-Score: {rfc_best_train_f1*100:.2f}%')
print(f'Testing F1-Score: {rfc_best_test_f1*100:.2f}%')

On the feature engineered test set, the Random Forest and Logistic Regression models both show relatively high accuracy (around 71% on the test set), with Random Forest slightly outperforming Logistic Regression in terms of both training and testing accuracy. XGBoost lags behind, with testing accuracy of 70.35%, indicating that it struggles slightly more to generalize compared to Random Forest and Logistic Regression. XGBoost has the highest F1-score (82.30%) on the test set, slightly outperforming Random Forest (82.10%) and Logistic Regression (81.58%). XGBoost is the best performing model overall due to its high recall (important for minimizing false negatives) and balanced F1-score (82.30%). This suggests that the ensemble methods (Random Forest and XGBoost) are best suited for the task of predicting homicide clearance, regardless of whether we use feature engineered data or not.







In [ ]:
importances = best_rfc_f.feature_importances_

# plot top 10 important features for rf model
feature_importance_df = pd.DataFrame({
    'Feature': X_features.columns,
    'Importance': importances
})

feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

feature_importance_df['Importance'] = feature_importance_df['Importance'].round(3)

top_n = 10
top_features = feature_importance_df.head(top_n)

plt.figure(figsize=(8, 6))
plt.barh(top_features['Feature'], top_features['Importance'])
plt.xlabel('Feature Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.title(f'Top {top_n} Feature Importances (Random Forest): Feature Engineered Data', fontsize=16)
plt.gca().invert_yaxis()
plt.show()

Similar to the XGBoost model, our engineered features seem to have high importances, with the Prior Month Homicide and the State PC1 variable being strong predictors in this random forest model. We also see that Firearm and Black_Male (the interaction term) are very important, showing that victim-specific characteristics as well as geographic variation (city level homicide rates and state socioeconomic factors) are important. This provides some evidence that our feature engineering strategies added value to the model, as they are important sources of variation.

However, feature engineering did not tremendously boost model performance, suggesting a need for more complicated feature engineering techniques, such as unsupervised algorithms like clustering or additional interaction terms.

## 5.6 Feedforward Neural Network



Although ensemble methods like XGBoost and Random Forest achieve high recall, they tend to over-predict solvable cases, leading to a high rate of false positives. To address this, we implement a simple feedforward neural network (FNN) to explore whether greater modeling flexibility can better capture complex interactions between victim, weapon, and agency characteristics. By learning more nuanced decision boundaries, the neural network may help improve precision without sacrificing recall, offering a potential path to reducing false positive rates in predicting homicide case solvability.

When building the neural network, we manually tested different parameters and structures to achieve the best F-1 score. We tried implementing Bayesian Optimization for hyper-paramater tuning, but found that it was too computationally intensive. Instead, we tested different approaches and structures and present the one that performs the best in this report.

While neural networks are often considered less interpretable compared to other models, their ability to capture complex, non-linear relationships makes them highly valuable for tasks where predictive power is prioritized. In this analysis, we chose to use a neural network because of its flexibility and capacity to model non-linear patterns in the data that oursimpler models might miss. While feature importance is important for understanding model decisions, we acknowledge that the neural network's "black-box" nature presents challenges in this regard. However, given we have gained an idea of the important features from our earlier models, we focused on leveraging the neural network's strengths without placing an undue emphasis on interpretability. Namely, if neural networks offer a significant gain in F-1 score, the trade-off with interpretability may be warranted in the sense that law enforcement agencies could use this model to prioritize resources to the harder to solve cases, without needing to know exactly what makes them harder to solve. Another key limitation with the neural network is that it requires a relatively large amount of data to perform well. Neural networks can easily overfit if the model architecture is too complex relative to the data size. Here, we have a large amount of data, but still need to balance this with model complexity.

### 5.6.1 Building the FNN Model

In [ ]:
# Define FNN class
class FNN(nn.Module):
    def __init__(self, input_dim, num_classes=2):
        super(FNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.type(torch.float32)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        outputs = self.fc3(x)
        return outputs

In [ ]:
# Wrap scaled data into Dataloaders
batch_size = 1024

X_train_scaled = X_train_scaled.astype(np.float32)
X_test_scaled = X_test_scaled.astype(np.float32)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fnn = FNN(input_dim=X_train_scaled.shape[1], num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fnn.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
acc_LIST_FNN = []
loss_LIST_FNN = []

for epoch in range(num_epochs):
    fnn.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = fnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    loss_LIST_FNN.append(running_loss / len(train_loader))
    acc_LIST_FNN.append(accuracy)

    print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, Accuracy={accuracy:.2f}%")


In [ ]:
# Plot Training Accuracy
epochs = range(1, len(acc_LIST_FNN) + 1)

plt.figure(figsize=(5, 3))
plt.plot(epochs, acc_LIST_FNN, marker='o')
plt.title("FNN Training Accuracy vs. Epochs")
plt.xlabel("Epochs")
plt.ylabel("Training Accuracy (%)")
plt.xticks(epochs)
plt.tight_layout()
plt.show()

This figure shows that the FNN became more accurate during each epoch of training.

In [ ]:
# Evaluate on Test Set
fnn.eval()
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = fnn(inputs)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

test_acc_FNN = (correct / total) * 100
print(f"Test Accuracy: {test_acc_FNN:.2f}%")

# Combine all preds and labels
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=3))


The feedforward neural network (FNN) achieves a Testing Accuracy of 72.10%, slightly higher than any of the tree-based models or logistic regression. Its F1-score for solvable cases (class 1) is 8176%, comparable to the F1-scores achieved by tuned logistic regression (81.77%), XGBoost (82.18%), and Random Forest (82.21%). The FNN model does achieve high recall (90.1%) and precision (74.8%) for solvable cases (class 1), making it effective if the sole objective were to prioritize identifying cases most likely to be cleared.

However, a deeper examination shows that the FNN model underperforms in detecting unsolvable cases (class 0). The recall for class 0 is only 31.4%, meaning that the model struggles to correctly identify when a homicide case is unlikely to be cleared. In contrast, tree-based models and logistic regression maintained a more balanced performance between classes, with stronger macro-averaged F1-scores.

Overall, while the FNN slightly outperforms other models in overall accuracy and class 1 performance, it sacrifices balanced detection between cleared and uncleared cases, which could limit its usefulness if identifying unsolvable cases early is a priority. Moreover, it does not lend itself to analyzing feature importance, which is a key advantage of the other models.

Despite experimenting with different learning rates, numbers of epochs, and architectural configurations, we found that the feedforward neural network consistently underperformed relative to the ensemble methods and logistic regression in terms of F-1 scores. One likely explanation is that, although the homicide clearance dataset is sizable, it may not provide sufficient signal relative to noise for a neural network to fully exploit. The structure of the data — with a moderate number of features and a relatively small number of informative interactions — may favor simpler models like XGBoost and Random Forests, which are better suited for tabular data with limited higher-order nonlinearities. As a result, while the neural network captures some variation in the data, it does not meaningfully outperform simpler, more structured models on this task.

### 5.6.2 Feature Engineering & FNN Model Performance

We also fit the FNN model to our feature engineered data, which has some more complex interaction terms, to see if feature engineering can improve the FNN model's performance

In [ ]:
X_train_tensor_f = torch.tensor(X_train_scaled_f, dtype=torch.float32)
y_train_tensor_f = torch.tensor(y_train_f, dtype=torch.long)
X_test_tensor_f = torch.tensor(X_test_scaled_f, dtype=torch.float32)
y_test_tensor_f = torch.tensor(y_test_f, dtype=torch.long)

train_dataset_f = TensorDataset(X_train_tensor_f, y_train_tensor_f)
test_dataset_f = TensorDataset(X_test_tensor_f, y_test_tensor_f)

train_loader_f = DataLoader(train_dataset_f, batch_size=batch_size, shuffle=True)
test_loader_f  = DataLoader(test_dataset_f, batch_size=batch_size, shuffle=False)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fnn_f= FNN(input_dim=X_train_scaled_f.shape[1], num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fnn.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
acc_LIST_FNN = []
loss_LIST_FNN = []

for epoch in range(num_epochs):
    fnn_f.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader_f:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = fnn_f(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    accuracy = 100 * correct / total
    loss_LIST_FNN.append(running_loss / len(train_loader))
    acc_LIST_FNN.append(accuracy)

    print(f"Epoch {epoch+1}: Loss={running_loss/len(train_loader):.4f}, Accuracy={accuracy:.2f}%")

In [ ]:
# Evaluate on Test Set
fnn_f.eval()
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader_f:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = fnn_f(inputs)
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

test_acc_FNN = (correct / total) * 100
print(f"Test Accuracy: {test_acc_FNN:.2f}%")
# Combine all preds and labels
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=3))


Again, we can see that feature engineering did not significantly improve the FNN model's performance.

## 5.7 Preferred Model



After comparing XGBoost, Random Forest, Logistic Regression, and the Feedforward Neural Network (FNN), Random Forest emerges as the preferred model for predicting homicide case clearance. Random Forest achieves the highest F1-score (82.21%), slightly outperforming XGBoost (82.18%) and Logistic Regression (81.77%). The F1-score is a crucial metric for balancing precision and recall, and in this case, Random Forest strikes a better balance, making it the most reliable in terms of overall performance. In addition to its higher F1-score, Random Forest also demonstrates a very strong recall (98.92% on the test set), which is vital for minimizing false negatives. While XGBoost has a slightly higher recall (99.48%), the difference is marginal. Although both models are tree-based, Random Forest benefits from a bagging approach, which reduces overfitting by averaging predictions across multiple independent trees, while XGBoost uses boosting to iteratively correct errors and improve prediction accuracy. These methods are particularly effective given the structured nature of the dataset, where non-linear relationships and interactions between features are important but not overly complex. Given the minimal difference between Random Forest and XGBoost, either model is well-suited to predict homicide clearance.

From an interpretability standpoint, Random Forest offers transparent feature importance metrics, allowing us to identify key drivers of case solvability, such as weapon type and state-level factors. This level of explainability is crucial for policy applications, where understanding the reasoning behind predictions is important. In contrast, the FNN lacks straightforward interpretability, making it harder to explain to non-technical stakeholders. XGBoost, while similar to Random Forest, may be preferred if a boosting approach is desired, though it performs almost identically in this analysis. Logistic Regression models offer more clear coefficient interpretation, with a direct impact on the odds of clearance, but it is still possible to judge feature importance in the tree-based models.

It is important to note that Logistic Regression, XGBoost, and Random Forest all performed similarly, suggesting that the relationships between the features in this dataset are relatively simple and close to linear. This allows Logistic Regression to capture these relationships effectively. On the other hand, the FNN, which excels at handling non-linear and complex interactions, underperformed, suggesting that the dataset does not have enough complexity to warrant the need for deep learning models. Overall, Random Forest strikes the best balance between predictive accuracy, model complexity, and interpretability, making it the most appropriate choice for predicting homicide case clearance in this analysis.

# Part 6. Conclusion

**Summary of Main Takeaways:**


*  Homicide clearance outcomes are more complex than anticipated. While we expected that victim demographics like race and sex would strongly influence case resolution, we were surprised by the strong impact of geographic features such as state and regional variations, indicating differences in investigative practices and reporting standards across the U.S. Additionally, firearm-related homicides were consistently less likely to be solved, likely due to fewer witnesses and less physical evidence, but the stark discrepancy in clearance based on weapon type was unexpected to us.

*   Our EDA revealed significant variations in clearance rates by age group, with teenage victims having some of the lowest clearance rates. Cases involving younger victims were more likely to be solved, possibly due to more public outcry. We also found evidence of racial disparities, with Black victims having lower clearance rates, highlighting the need for law enforcement to ensure equitable investigations. Overall, our takeaway was that EDA is extremely important to shaping the types of hypotheses you test, and can point immeditaely to meaninful relationships.

* In terms of modeling implications for stakeholders, the best model was the tuned Random Forest Classifier, achieving a Testing F1-Score of approximately 82.2% and Testing Recall of 98.9%. The model’s ability to generate transparent feature importance metrics is particularly valuable for law enforcement agencies, as it allows them to identify the key factors that drive case solvability, such as weapon type and state-level characteristics.

* Although we had hoped that neural networks could yield even better results, the structure and size of the dataset ultimately favored tree-based/ensemble models over deep learning approaches. While neural networks can capture complex interactions, our dataset apparently did not present sufficient complexity to warrant the use of deep learning models. This highlights the importance of selecting the right model for the dataset.

* While feature importance metrics provide valuable insights, both the Random Forest and XGBoost models are less interpretable that the Logistic Regression model, which performed similarly to the tree-based models. Logistic Regression offers clear coefficient interpretation but lacks the complexity to capture non-linear relationships effectively. In some cases, however, logistic regression may be the better choice, especially when explaning the important predictors to non-technical stakehodlers.

* Overall, our analysis showed that regional disparities are important predictors of homicide clearance. This suggests that some states should allocate more resources to invetigating homicides. We also found evidence of significant racial disparities, suggesting the need for training programs to ensure officers investigate cases involving Black and white victims more equitably. Law enforcement agencies could use our model to identify which cases are most likely to be solved, devote limited manpower to these, and allocate more resources towards homicides that are less likely to be solved. This resource allocation is important because unsolved homicides mean that murderers go free-- endangering the community.

* The main challenges we faced was deciding how to conduct feature engineering based on the trends we saw in EDA, and hyperparameter tuning with only marginal improvements in performance. It was also difficult to prioritize the EDA visuals and hypothesis tests that would be most impactful for stakeholders (law enforcement agencies).


**Future Directions for Exploration:**

* Joining the current dataset with additional case-specific information — such as more granular details about investigative practices, officer counts, or forensic resources — could meaningfully improve model complexity and predictive power.

* Experimenting with deeper or more regularized neural networks (e.g., adding dropout, batch normalization) might yield improvements if larger datasets become available. Including data from recent years may also help us learn more about trends in homicide.

* Performing more advanced feature engineering on existing categorical variables, such as clustering weapon types or agency types, to capture higher-level patterns that individual dummies may miss.

* Conducting statistical significance testing on logistic regression coefficients to better understand which predictors are robust across different model formulations.

**Overall Experience and Team Reflection:**

Working on this project was a valuable experience that allowed us to apply many of the skills we developed throughout the semester — from exploratory data analysis to advanced machine learning modeling. It was interesting to observe how different modeling strategies performed on the same dataset and how certain intuitive hypotheses (e.g., about the dominance of victim characteristics like race and sex) were challenged by empirical results. Although it was frustrating to debug model issues and tune hyperparameters with minimal improvements in terms of performance, we now have a much stronger appreciation for why predictive modeling in real-world data science requires constant experimentation as well as EDA to guide the features and tests you engineer.

We also want to thank the CIS 5450 instructional team for their support and guidance throughout the semester!